In [38]:
import os
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Optional
import logging
import traceback
from pathlib import Path

from Classes.SimplifiedChatbots import ChatGPT, BigSummarizerGPT
from Classes.DocumentProcessor import DocumentProcessor, CitationProcessor
from Utils.Helpers import *

class PoliticianAnalyzer:
    """
    A class for analyzing media coverage of politicians.
    Implements a hybrid approach that leverages core components
    while adding politician-specific analysis.
    """
    
    def __init__(self, 
                 politician_name: str, 
                 articles: List[Dict], 
                 general_folder: str,
                 region: str = None,
                 political_party: str = None,
                 language: str = "English",
                 force_reprocess: bool = False):
        """
        Initialize the politician analyzer.
        
        Args:
            politician_name (str): Name of the politician being analyzed
            articles (List[Dict]): List of preprocessed articles
            general_folder (str): Base path for output files
            region (str, optional): Geographic region of politician
            political_party (str, optional): Political party affiliation
            language (str, optional): Output language for analysis
            force_reprocess (bool, optional): Whether to force reprocessing
        """
        self.politician_name = politician_name
        self.articles = articles
        self.general_folder = general_folder
        self.region = region
        self.political_party = political_party
        self.language = language
        self.force_reprocess = force_reprocess
        
        # Set up directories
        self.setup_directories()
        
        # Initialize processing status
        self.has_basic_sentiment = False
        self.has_narratives = False
        self.has_policy_positions = False
        self.has_stakeholders = False
        self.has_entities = False
        
        # Check existing processing status
        self.check_processing_status()
    
    def setup_directories(self):
        """Create necessary directory structure for outputs."""
        # Main output directories
        os.makedirs(os.path.join(self.general_folder, "Outputs", "CompiledOutputs"), exist_ok=True)
        os.makedirs(os.path.join(self.general_folder, "Outputs", "PoliticianAnalysis"), exist_ok=True)
        os.makedirs(os.path.join(self.general_folder, "Outputs", "Visualizations"), exist_ok=True)
        
        # Specialized directories
        os.makedirs(os.path.join(self.general_folder, "Outputs", "PoliticianAnalysis", "PolicyPositions"), exist_ok=True)
        os.makedirs(os.path.join(self.general_folder, "Outputs", "PoliticianAnalysis", "Narratives"), exist_ok=True)
        os.makedirs(os.path.join(self.general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders"), exist_ok=True)
        os.makedirs(os.path.join(self.general_folder, "Outputs", "PoliticianAnalysis", "Timeline"), exist_ok=True)
    
    def check_processing_status(self):
        """Check for existing processed data to avoid redundant processing."""
        politician_data_path = os.path.join(
            self.general_folder, "Outputs", "CompiledOutputs", 
            f"PoliticianAnalysis_{self.politician_name.replace(' ', '_')}.json"
        )
        
        if os.path.exists(politician_data_path) and not self.force_reprocess:
            try:
                with open(politician_data_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                self.has_basic_sentiment = data.get('has_basic_sentiment', False)
                self.has_narratives = data.get('has_narratives', False)
                self.has_policy_positions = data.get('has_policy_positions', False)
                self.has_stakeholders = data.get('has_stakeholders', False)
                self.has_entities = data.get('has_entities', False)
                
                # If we have processed data, update our articles with it
                if 'articles' in data:
                    self.articles = data['articles']
                    logging.info(f"Loaded {len(self.articles)} preprocessed articles")
            except Exception as e:
                logging.error(f"Error loading preprocessing status: {str(e)}")
                logging.error(traceback.format_exc())
    
    def save_processing_status(self):
        """Save current processing status and articles to avoid reprocessing."""
        politician_data_path = os.path.join(
            self.general_folder, "Outputs", "CompiledOutputs", 
            f"PoliticianAnalysis_{self.politician_name.replace(' ', '_')}.json"
        )
        
        try:
            data = {
                'politician_name': self.politician_name,
                'has_basic_sentiment': self.has_basic_sentiment,
                'has_narratives': self.has_narratives,
                'has_policy_positions': self.has_policy_positions,
                'has_stakeholders': self.has_stakeholders,
                'has_entities': self.has_entities,
                'last_updated': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'articles': self.articles
            }
            
            with open(politician_data_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2)
                
            logging.info(f"Saved processing status to {politician_data_path}")
        except Exception as e:
            logging.error(f"Error saving processing status: {str(e)}")
            logging.error(traceback.format_exc())
    
    def preprocess_articles(self):
        """
        Preprocess articles for analysis, including:
        - Extracting basic sentiment
        - Adding timestamps for chronological analysis
        - Deduplicating articles
        """
        logging.info(f"Preprocessing {len(self.articles)} articles")
        
        if self.has_basic_sentiment and not self.force_reprocess:
            logging.info("Using existing sentiment analysis")
            return self.articles
        
        system_prompt = f"""You are a helpful assistant. Your role is to assess the overall tone of a news article, specifically focusing on the article's content about {self.politician_name}."""
        
        for article in self.articles:
            # Add timestamp for chronological analysis
            if 'date' in article:
                try:
                    date_object = datetime.strptime(article['date'], '%B %d, %Y')
                    article['timestamp'] = date_object.timestamp()
                except (ValueError, TypeError):
                    logging.warning(f"Could not parse date: {article.get('date')}")
                    article['timestamp'] = 0
            else:
                article['timestamp'] = 0
            
            # Add sentiment analysis if not already present
            if 'tone' not in article or not article['tone'] or self.force_reprocess:
                chatbot = ChatGPT(
                    system_prompt=system_prompt,
                    model_name="gpt-4o-mini",
                    temperature=0,
                    max_tokens=50,
                )
                question = f"""
Please assess the tone of the news article with specific regard to: {self.politician_name}. Focus on how the article portrays {self.politician_name} in terms of actions, decisions, statements, and character. The tone should be categorized as one of the following:

Positive: The article reflects well on {self.politician_name}, highlighting favorable aspects.
Neutral: The article presents information about {self.politician_name} in a balanced, objective manner without any strong positive or negative bias.
Negative: The article is critical of {self.politician_name}, highlighting challenges, controversies, failures, or unfavorable aspects.

Provide the final tone assessment as "Positive," "Neutral," or "Negative."

Here is the article content: {article['content']}

Your output should solely be one of these three words based on your assessment: "Positive", "Neutral", or "Negative". Nothing else
                """
                response = chatbot.ask(question)
                print(response)
                article['tone'] = response.strip()
            
            # Add sentiment score if not already present
            if 'sentiment_score' not in article or article['sentiment_score'] is None or self.force_reprocess:
                chatbot = ChatGPT(
                    system_prompt=system_prompt,
                    model_name="gpt-4o-mini",
                    temperature=0,
                    max_tokens=50,
                )
                question = f"""
Perform a detailed sentiment analysis of the article provided below, focusing exclusively on how it describes {self.politician_name}. Your task is to evaluate the overall sentiment expressed in the article regarding {self.politician_name} by carefully considering the tone, language, context, and any descriptive cues related to {self.politician_name}.

Guidelines for Scoring:

-5: The article is extremely critical and conveys a highly negative sentiment toward {self.politician_name}.
-4: The article offers notable criticism, highlighting significant negative aspects of {self.politician_name}.
-3: The sentiment is moderately negative, with clear indications of disapproval of {self.politician_name}.
-2: Some negative remarks about {self.politician_name} are present, though they are not predominant.
-1: The article shows mild negativity or slight disapproval of {self.politician_name}.
0: The portrayal of {self.politician_name} is neutral with no significant positive or negative sentiment.
1: The article exhibits slight positive sentiment or mild approval of {self.politician_name}.
2: The tone is moderately positive, suggesting a favorable view of {self.politician_name}.
3: The article is largely favorable, displaying clear positive sentiment about {self.politician_name}.
4: The content strongly praises {self.politician_name}.
5: The article is exceptionally complimentary, demonstrating an extremely positive sentiment about {self.politician_name}.

Instructions:
Analyze the language, tone, and context used in the article with respect to {self.politician_name}.
Based solely on the observations, assign one sentiment score from the list: [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5].
Your final output must be exactly one of these values and nothing else.
Article Content: {article['content']}

Only output one of these values: [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5], and nothing else.
                """
                response = chatbot.ask(question)
                print(response)
                try:
                    sentiment_score = int(response.strip())
                    if sentiment_score < -5 or sentiment_score > 5:
                        raise ValueError("Score out of range")
                    article['sentiment_score'] = sentiment_score
                except (ValueError, TypeError):
                    logging.warning(f"Invalid sentiment score: {response}. Using 0 as default.")
                    article['sentiment_score'] = 0
        
        # Sort articles chronologically
        self.articles = sorted(self.articles, key=lambda x: x.get('timestamp', 0))
        
        self.has_basic_sentiment = True
        self.save_processing_status()
        
        return self.articles
    
    def analyze_narratives(self):
        """
        Identify recurring narratives and frames used to describe the politician.
        Returns dictionary of narrative categories and their descriptions.
        """
        logging.info("Beginning narrative analysis")
        
        if self.has_narratives and not self.force_reprocess:
            logging.info("Using existing narrative analysis")
            return self._get_narratives_from_articles()
        
        # First generate one-sentence descriptions for each article
        system_prompt = f"""You are a political media analyst. Your role is to describe in one single sentence what a given news media article says about {self.politician_name}."""
        compiled_sentences = ""
        
        for article in self.articles:
            if 'one_sentence_description' not in article or self.force_reprocess:
                chatbot = ChatGPT(
                    system_prompt=system_prompt,
                    model_name="gpt-4o-mini",
                    temperature=0,
                    max_tokens=350,
                )
                question = f"""
Please write a single sentence summarizing this article's portrayal of {self.politician_name}. Focus on how the politician is framed, what actions or positions are highlighted, and the overall narrative about them.

Article: {article['content']}
                """
                response = chatbot.ask(question)
                print(response)
                article['one_sentence_description'] = response
            
            compiled_sentences += article['one_sentence_description'] + "\n"
        
        # Identify key narrative frames
        system_prompt = f"""You are a political communication expert. Your role is to identify the primary recurring narratives and frames used to portray {self.politician_name} across multiple media articles."""
        
        chatbot = ChatGPT(
            system_prompt=system_prompt,
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=1500,
        )
        
        question = f"""
Based on these article summaries, identify the main recurring narratives or frames used to portray {self.politician_name} in the media.
Create 5-8 distinct narrative categories that represent specific framing patterns, recurring stories, or thematic approaches to covering this politician.

Article summaries:
{compiled_sentences}

For each narrative category:
1. Give it a specific name that describes the frame or narrative approach
2. Provide a detailed description of what this narrative entails, how it portrays the politician
3. Focus on identifying narrative patterns rather than just policy areas

Format your response exactly as follows:

CATEGORY: [Narrative/Frame Name]
DESCRIPTION: [Detailed explanation of the narrative/frame and how it portrays the politician]

Ensure categories are distinct and capture different aspects of media portrayal.
"""
        
        categories_response = chatbot.ask(question)
        print(categories_response)
        
        # Parse the narrative categories
        categories_data = []
        current_category = None
        current_description = None
        
        for line in categories_response.split('\n'):
            if line.startswith('CATEGORY:'):
                if current_category is not None:
                    categories_data.append({
                        'category': current_category.strip(),
                        'description': current_description.strip() if current_description else '',
                        'articles': []
                    })
                current_category = line.replace('CATEGORY:', '').strip()
                current_description = None
            elif line.startswith('DESCRIPTION:'):
                current_description = line.replace('DESCRIPTION:', '').strip()
        
        # Add the last category
        if current_category is not None:
            categories_data.append({
                'category': current_category,
                'description': current_description if current_description else '',
                'articles': []
            })
        
        # Categorize each article into a narrative category
        for article in self.articles:
            classification_prompt = ""
            for category in categories_data:
                classification_prompt += f"\n{category['category']}: {category['description']}"
            
            chatbot = ChatGPT(
                model_name="gpt-4o-mini",
                temperature=0,
                max_tokens=200
            )
            
            classification = chatbot.ask(
                f"""
Given these narrative categories for {self.politician_name}:
{classification_prompt}

Classify this article into one of these categories. Choose the most appropriate narrative frame based on the descriptions provided.

Article content:
{article['content']}

Only output the exact category name that best matches this article. Your output should be just the category name and nothing else.
                """
            )
            print(classification)
            
            # Find the matching category
            article['narrative_category'] = None
            for category in categories_data:
                if classification.strip() == category['category'].strip():
                    article['narrative_category'] = category['category']
                    category['articles'].append(article)
                    break
            
            # If no match was found, use the first category as a fallback
            if article['narrative_category'] is None and categories_data:
                article['narrative_category'] = categories_data[0]['category']
                categories_data[0]['articles'].append(article)
        
        # Save narrative frames as a separate file
        narrative_frames_path = os.path.join(
            self.general_folder, "Outputs", "PoliticianAnalysis", "Narratives",
            f"NarrativeFrames_{self.politician_name.replace(' ', '_')}.json"
        )
        
        with open(narrative_frames_path, 'w', encoding='utf-8') as f:
            json.dump(categories_data, f, indent=2)
        
        self.has_narratives = True
        self.save_processing_status()
        
        return categories_data
    
    def _get_narratives_from_articles(self):
        """Helper method to reconstruct narrative categories from article data."""
        narrative_frames_path = os.path.join(
            self.general_folder, "Outputs", "PoliticianAnalysis", "Narratives",
            f"NarrativeFrames_{self.politician_name.replace(' ', '_')}.json"
        )
        
        if os.path.exists(narrative_frames_path):
            with open(narrative_frames_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        
        # If file doesn't exist but articles have narrative categories,
        # reconstruct from article data
        categories = {}
        for article in self.articles:
            if 'narrative_category' in article:
                category = article['narrative_category']
                if category not in categories:
                    categories[category] = {
                        'category': category,
                        'description': '',  # We don't have descriptions
                        'articles': []
                    }
                categories[category]['articles'].append(article)
        
        return list(categories.values())
    
    def analyze_policy_positions(self, force_reprocess=False):
        """
        Extract and analyze how the politician's policy positions are portrayed using BigSummarizerGPT
        to process large volumes of text, with an additional clustering step to consolidate similar policy areas.
        Returns dictionary of policy areas and associated data.
        """
        logging.info("Beginning policy position analysis")

        # Use either the instance-level or method-level force_reprocess
        local_force_reprocess = force_reprocess or self.force_reprocess

        if self.has_policy_positions and not local_force_reprocess:
            logging.info("Using existing policy position analysis")
            policy_positions_path = os.path.join(
                self.general_folder, "Outputs", "PoliticianAnalysis", "PolicyPositions",
                f"PolicyPositions_{self.politician_name.replace(' ', '_')}.json"
            )

            if os.path.exists(policy_positions_path):
                with open(policy_positions_path, 'r', encoding='utf-8') as f:
                    policy_clusters = json.load(f)
                    # Generate the MD output file even if we're using cached data
                    self._generate_policy_clusters_md(policy_clusters)
                    return policy_clusters
            else:
                logging.warning("Policy positions marked as processed but file not found")

        # Compile all article contents for analysis
        temp_compiled_path = os.path.join(
            self.general_folder, "Outputs", "CompiledOutputs", 
            f"CompiledArticles_{self.politician_name.replace(' ', '_')}.md"
        )

        compiled_content = ""
        for article in self.articles:
            metadata = f"""
    Title: {article.get('title', 'Untitled')}
    Media outlet: {article.get('media_outlet', 'Unknown')}
    Date: {article.get('date', 'Unknown')}
    """
            compiled_content += "\n\n" + "---" + metadata + article.get('content', '') + "\n\n" + metadata + "---\n\n"

        # Save compiled content for BigSummarizerGPT
        with open(temp_compiled_path, "w", encoding='utf-8') as f:
            f.write(compiled_content)

        # STEP 1: Identify initial policy areas using BigSummarizerGPT
        system_prompt = f"""You are a political policy analyst. Your role is to identify the main policy areas discussed in relation to {self.politician_name} across multiple media articles."""

        bigbot = BigSummarizerGPT(
            system_prompt=system_prompt,
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=1500
        )

        policy_areas_prompt = f"""
Based on the media coverage, identify the main policy areas where {self.politician_name}'s positions, statements, or actions are discussed.
Create a comprehensive list of distinct policy areas that represent the key domains of political discussion.
Be specific and provide details - we will cluster these together in a later step.

For each policy area:
1. Give it a clear, specific name (e.g., "Carbon Pricing Policy" rather than just "Climate Policy")
2. Provide a brief description of what this policy area encompasses
3. Focus on areas where the politician's positions or actions are specifically mentioned

Format your response exactly as follows:

POLICY_AREA: [Policy Area Name]
DESCRIPTION: [Brief description of what this policy area covers]

Aim to identify all distinct policy areas without concern for the total number at this stage.
    """

        policy_areas_response = bigbot.ask(policy_areas_prompt, temp_compiled_path)
        print("Initial policy areas identified:")
        print(policy_areas_response)

        # Parse initial policy areas
        initial_policy_areas = []
        current_area = None
        current_description = None

        for line in policy_areas_response.split('\n'):
            if line.startswith('POLICY_AREA:'):
                if current_area is not None:
                    initial_policy_areas.append({
                        'policy_area': current_area.strip(),
                        'description': current_description.strip() if current_description else ''
                    })
                current_area = line.replace('POLICY_AREA:', '').strip()
                current_description = None
            elif line.startswith('DESCRIPTION:'):
                current_description = line.replace('DESCRIPTION:', '').strip()

        # Add the last policy area
        if current_area is not None:
            initial_policy_areas.append({
                'policy_area': current_area.strip(),
                'description': current_description.strip() if current_description else ''
            })

        print(f"Parsed {len(initial_policy_areas)} initial policy areas")

        # STEP 2: Cluster policy areas into broader categories (6-9 clusters)
        chatbot = ChatGPT(
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=2000
        )

        clustering_prompt = f"""
I've identified {len(initial_policy_areas)} specific policy areas related to {self.politician_name}. 
I need to cluster these into 6-9 broader policy categories that are still descriptive and meaningful.

For example, specific areas like "Carbon Pricing", "Biodiversity Restoration", and "Renewable Energy Subsidies" 
might be clustered into a broader "Environment & Climate Policy" category.

Here are the specific policy areas:

{json.dumps([{'name': area['policy_area'], 'description': area['description']} for area in initial_policy_areas], indent=2)}

Please create 6-9 broader policy clusters by grouping these specific areas.
For each cluster:
1. Provide a descriptive name that captures the theme
2. Make an exhaustive and extensive list of which specific policy areas belong to this cluster (BUT AVOID DUPLICATING SYNONYM POLICIES)
3. Write a comprehensive description of the cluster that incorporates elements from its component areas

Format your response as follows:

CLUSTER: [Cluster Name]
COMPONENTS: [List of specific policy areas that belong to this cluster, without duplicating synonym policies]
DESCRIPTION: [Comprehensive description of this policy cluster]

Ensure that every specific policy area is assigned to exactly one cluster, and that the clusters are balanced and meaningful.
    """

        clustering_response = chatbot.ask(clustering_prompt)
        print("Policy clustering result:")
        print(clustering_response)

        # Parse the clustered policy areas
        policy_clusters = []
        current_cluster = None
        current_components = None
        current_description = None

        for line in clustering_response.split('\n'):
            line = line.strip()
            if not line:
                continue

            # Updated parsing for "CLUSTER:" format
            if "CLUSTER:" in line:
                if current_cluster is not None:
                    policy_clusters.append({
                        'policy_area': current_cluster.strip(),
                        'components': current_components if current_components else [],
                        'description': current_description.strip() if current_description else '',
                        'positions': [],
                        'sentiment': {
                            'average': 0,
                            'positive': 0,
                            'neutral': 0,
                            'negative': 0
                        }
                    })
                # Extract cluster name, handling both "CLUSTER: X" and "**CLUSTER:** X" formats
                current_cluster = line.split("CLUSTER:")[1].replace('*', '').strip()
                current_components = []
                current_description = None

            # Handle components section - note the response is using bullet points
            elif "COMPONENTS:" in line or line.strip().startswith('- '):
                if "COMPONENTS:" in line:
                    # Skip the header line
                    continue
                elif line.strip().startswith('- '):
                    # This is a component item - add to components list
                    component = line.strip().replace('- ', '').strip()
                    if current_components is not None:
                        current_components.append(component)

            # Handle description section
            elif "DESCRIPTION:" in line:
                # Start capturing description
                current_description = line.replace("DESCRIPTION:", "").strip().replace('*', '')
            elif current_description is not None:
                # Continue appending to description
                current_description += " " + line.strip()

        # Add the last cluster if it exists
        if current_cluster is not None:
            policy_clusters.append({
                'policy_area': current_cluster.strip(),
                'components': current_components if current_components else [],
                'description': current_description.strip() if current_description else '',
                'positions': [],
                'sentiment': {
                    'average': 0,
                    'positive': 0,
                    'neutral': 0,
                    'negative': 0
                }
            })

        print(f"Created {len(policy_clusters)} policy clusters")

        # STEP 3: Extract positions for each policy cluster using BigSummarizerGPT
        for policy_cluster in policy_clusters:
            # Create a comma-separated list of all component policy areas
            components_str = ", ".join(policy_cluster['components'])

            bigbot = BigSummarizerGPT(
                system_prompt=system_prompt,
                model_name="gpt-4o-mini",
                temperature=0,
                max_tokens=1500
            )

            positions_prompt = f"""
Extract comprehensive information about {self.politician_name}'s positions, statements, or actions related to {policy_cluster['policy_area']}.
Among others, This cluster includes the following specific policy areas: {components_str}

For each relevant mention across all articles, provide:
1. A direct quote or paraphrase of the position/statement (if available)
2. The context in which it was mentioned
3. The date of the article
4. The media outlet

Format each position as a bullet point starting with a dash (-).
Each bullet point should be self-contained and include all the information above.
For each bullet point, include references to the specific articles where each position was mentioned. Reference the media outlet, author and date in [Media outlet, Austhor, date].
If no information is found for this policy area, output: "- No specific positions found on {policy_cluster['policy_area']}."
    """

            positions_response = bigbot.ask(positions_prompt, temp_compiled_path)
            print(f"Extracted positions for {policy_cluster['policy_area']}:")
            print(positions_response)

            # Parse bullet points
            bullet_points = re.findall(r'(?:^|\n)- .+?(?=\n-|\n\n|$)', positions_response, re.DOTALL)
            bullet_points = [point.strip() for point in bullet_points]

            # Add to policy cluster data
            policy_cluster['positions'] = bullet_points
            policy_cluster['position_count'] = len(bullet_points)

            # STEP 4: Generate a concise analysis of each policy cluster
            analysis_prompt = f"""
Based on the extracted positions, create a concise, well-structured analysis of {self.politician_name}'s stance on {policy_cluster['policy_area']}.

Policy cluster: {policy_cluster['policy_area']}
Component policy areas: {components_str}
Cluster description: {policy_cluster['description']}

Extracted positions and statements:
{positions_response}

Please provide a comprehensive analysis that:
1. Summarizes {self.politician_name}'s overall position or approach to this policy area
2. Identifies key themes, consistencies, or contradictions in their stance
3. Notes how their position has evolved over time (if applicable)
4. References specific statements or actions, citing the media sources
5. Analyzes how media outlets frame their position

Format your analysis as a concise, well-structured summary of 3-4 paragraphs.
Focus on providing a specific positions and quotes which clearly illustrate {self.politician_name}'s overall position on {policy_cluster['policy_area']}.
Adopt a neutral writing style, your output should not be overly favourable or defavourable to {self.politician_name}. It should be objective.
Include specific references to media sources in [brackets] to support your points.
    """

            analysis_chatbot = ChatGPT(
                model_name="chatgpt-4o-latest",
                temperature=0,
                max_tokens=1900
            )

            policy_analysis = analysis_chatbot.ask(analysis_prompt)
            print(f"Analysis for {policy_cluster['policy_area']}:")
            print(policy_analysis)
            policy_cluster['analysis'] = policy_analysis

        # Save policy positions
        policy_positions_path = os.path.join(
            self.general_folder, "Outputs", "PoliticianAnalysis", "PolicyPositions",
            f"PolicyPositions_{self.politician_name.replace(' ', '_')}.json"
        )

        with open(policy_positions_path, 'w', encoding='utf-8') as f:
            json.dump(policy_clusters, f, indent=2)

        self.has_policy_positions = True
        self.save_processing_status()

        # Generate the nicely formatted markdown output file
        self._generate_policy_clusters_md(policy_clusters)

        return policy_clusters

    def _generate_policy_clusters_md(self, policy_clusters):
        """
        Generate a nicely formatted markdown file for the policy analysis results.

        Args:
            policy_clusters (List[Dict]): The policy clusters data
        """
        logging.info(f"Generating policy clusters markdown file for {self.politician_name}")

        # Create the output path
        output_dir = os.path.join(self.general_folder, "Outputs", "PoliticianAnalysis", "PolicyPositions")
        os.makedirs(output_dir, exist_ok=True)

        output_path = os.path.join(
            output_dir,
            f"PolicyClusters_{self.politician_name.replace(' ', '_')}.md"
        )

        # Get the date of analysis
        current_date = datetime.now().strftime("%B %d, %Y")

        # Count total positions across all clusters
        total_positions = sum(len(cluster.get('positions', [])) for cluster in policy_clusters)

        # Generate chart first to ensure it exists for the document
        try:
            chart_path = self._generate_policy_distribution_chart(policy_clusters)
            chart_relative_path = os.path.relpath(
                chart_path, 
                os.path.dirname(output_path)
            )
        except Exception as e:
            logging.warning(f"Could not generate policy distribution chart: {str(e)}")
            chart_relative_path = f"../../../Outputs/Visualizations/PolicyDistribution_{self.politician_name.replace(' ', '_')}.png"

        # Generate introduction using AI
        try:
            introduction = self._generate_introduction(policy_clusters)
        except Exception as e:
            logging.warning(f"Could not generate introduction: {str(e)}")
            introduction = f"This document presents an analysis of {self.politician_name}'s policy positions based on media coverage, organized into {len(policy_clusters)} key policy clusters. The analysis synthesizes information from {total_positions} distinct policy statements or actions reported in media sources."

        # Start building the markdown content
        md_content = f"""# Policy Position Analysis: {self.politician_name}

## Overview

{introduction}

## Policy Mentions Distribution

The following chart shows the distribution of {self.politician_name}'s policy mentions across major policy areas:

![Policy Mentions Distribution]({chart_relative_path})

<div style="page-break-after: always;"></div>

## Table of Contents

"""

        # Add table of contents
        for i, cluster in enumerate(policy_clusters, 1):
            md_content += f"{i}. [{cluster['policy_area']}](#{cluster['policy_area'].lower().replace(' ', '-')})\n"

        md_content += "\n---\n\n"

        # Add detailed cluster analyses
        for i, cluster in enumerate(policy_clusters, 1):
            position_count = len(cluster.get('positions', []))
            component_count = len(cluster.get('components', []))

            # Create section anchor
            anchor = cluster['policy_area'].lower().replace(' ', '-')
            new_section = f"""<a id="{anchor}"></a>
## {i}. {cluster['policy_area']}

### Overview
{cluster['description']}

**Media Coverage**: {position_count} related mentions of positions/statements identified

### Positions and Statements
{cluster['analysis']}

"""
            new_section = new_section.replace('---', '')
            md_content += "\n"
            md_content += new_section
            md_content += "\n"

        # Generate conclusion using AI
        try:
            conclusion = self._generate_conclusion(policy_clusters)
        except Exception as e:
            logging.warning(f"Could not generate conclusion: {str(e)}")
            conclusion = f"This analysis presents a comprehensive overview of {self.politician_name}'s policy positions across {len(policy_clusters)} major policy areas. The data is derived from media coverage and provides insights into how {self.politician_name}'s political stance is portrayed in public discourse."

        # Add conclusion section
        md_content += f"""## Conclusion

{conclusion}
"""

        try:
            # Import the processor if not already imported
            from Classes.DocumentProcessor import CitationProcessor
            processor = CitationProcessor()
            md_content = processor.process_citations_in_markdown(md_content)
            logging.info("Successfully processed citations to superscript format")
        except Exception as e:
            logging.error(f"Error processing citations: {str(e)}")
            # Continue with unprocessed content if citation processing fails

        # Write the markdown file
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(md_content)

        logging.info(f"Policy clusters markdown file generated at {output_path}")
        return output_path

    def _generate_introduction(self, policy_clusters):
        """Generate an AI-powered introduction for the policy analysis."""
        try:
            chatbot = ChatGPT(
                model_name="chatgpt-4o-latest",
                temperature=0,
                max_tokens=1000
            )

            # Prepare prompt with summarized policy data
            policy_summary = "\n".join([
                f"- {cluster['policy_area']}: {len(cluster.get('positions', []))} mentions, covering: {', '.join(cluster.get('components', []))[:100]}..."
                for cluster in policy_clusters
            ])

            prompt = f"""
Generate a concise introduction (2-3 paragraphs) for a policy position analysis document about {self.politician_name}.
Include:
1. A brief contextual overview of {self.politician_name}'s role
2. The significance of analyzing their policy positions
3. A summary of the key policy areas covered in the analysis
4. The methodology (analysis of media coverage)

Policy clusters analyzed:
{policy_summary}

The introduction should be informative, neutral, and professional in tone.
    """

            introduction = chatbot.ask(prompt)
            return introduction

        except Exception as e:
            logging.error(f"Error generating introduction: {str(e)}")
            raise

    def _generate_conclusion(self, policy_clusters):
        """Generate an AI-powered conclusion for the policy analysis."""
        try:
            chatbot = ChatGPT(
                model_name="chatgpt-4o-latest",
                temperature=0,
                max_tokens=1000
            )

            # Get top 3 policy areas by position count
            sorted_clusters = sorted(
                policy_clusters, 
                key=lambda x: len(x.get('positions', [])), 
                reverse=True
            )
            top_clusters = sorted_clusters[:min(3, len(sorted_clusters))]

            top_summary = "\n".join([
                f"- {cluster['policy_area']}: {len(cluster.get('positions', []))} mentions"
                for cluster in top_clusters
            ])

            prompt = f"""
Generate a concise conclusion (2-3 paragraphs) for a policy position analysis document about {self.politician_name}.
Include:
1. A summary of the most significant policy areas based on media coverage
2. Key patterns or themes observed across policy areas
3. The broader implications for understanding {self.politician_name}'s political stance
4. The value of this analysis for stakeholders and the public

Top policy areas by media mentions:
{top_summary}

The conclusion should be informative, neutral, and professional in tone.
    """

            conclusion = chatbot.ask(prompt)
            return conclusion

        except Exception as e:
            logging.error(f"Error generating conclusion: {str(e)}")
            raise
    
    def _generate_policy_distribution_chart(self, policy_clusters):
        """
        Generate a visualization of policy distribution across clusters.

        Args:
            policy_clusters (List[Dict]): The policy clusters data
        """
        viz_dir = os.path.join(self.general_folder, "Outputs", "Visualizations")
        os.makedirs(viz_dir, exist_ok=True)

        # Extract data for visualization
        labels = [cluster['policy_area'] for cluster in policy_clusters]
        sizes = [len(cluster.get('positions', [])) for cluster in policy_clusters]

        # Only proceed if we have data
        if sum(sizes) == 0:
            logging.warning("No policy positions to visualize")
            return

        # Convert to pandas Series for compatibility with create_professional_pie
        from pandas import Series
        data = Series(sizes, index=labels)

        # Create a professional pie chart using the helper function
        try:
            from Utils.Helpers import create_professional_pie, create_custom_colormap, get_text_color
            fig = create_professional_pie(
                data=data,
                title=f"{self.politician_name}'s Policy Mentions Distribution",
                figsize=(15, 10)
            )
        except ImportError:
            # Fallback to standard matplotlib pie chart if helper functions not available
            fig = plt.figure(figsize=(15, 10))
            plt.pie(sizes, labels=None, autopct='%1.1f%%', startangle=90, 
                    shadow=False, explode=[0.05] * len(sizes))
            plt.axis('equal')
            plt.title(f"{self.politician_name}'s Policy Mentions Distribution", fontsize=16, pad=20)

            # Add a legend with percentages
            percentages = [100 * size / sum(sizes) for size in sizes]
            legend_labels = [f"{label} ({size} mentions, {percentage:.1f}%)" 
                             for label, size, percentage in zip(labels, sizes, percentages)]

            plt.legend(legend_labels, loc="best", bbox_to_anchor=(1, 0.5), fontsize=10)
            plt.tight_layout()

        # Save the chart
        chart_path = os.path.join(viz_dir, f"PolicyDistribution_{self.politician_name.replace(' ', '_')}.png")
        plt.savefig(chart_path, bbox_inches='tight', dpi=300)
        plt.close()

        logging.info(f"Policy distribution chart saved to {chart_path}")
        return chart_path
    
    def analyze_stakeholders(self):
        """
        Analyze the politician's relationships with stakeholders mentioned in the articles.
        Identifies people and organizations relevant to the politician, categorizes them,
        and generates comprehensive stakeholder analysis.

        Returns:
            dict: Structured stakeholder analysis with categorization and relationship mapping
        """
        logging.info(f"Beginning stakeholder analysis for {self.politician_name}")

        # Check if we already have processed stakeholder data
        stakeholder_path = os.path.join(
            self.general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders",
            f"StakeholderAnalysis_{self.politician_name.replace(' ', '_')}.json"
        )

        if os.path.exists(stakeholder_path) and not self.force_reprocess:
            logging.info("Loading existing stakeholder analysis")
            with open(stakeholder_path, 'r', encoding='utf-8') as f:
                return json.load(f)

        # Step 1: Extract stakeholders from each article
        all_stakeholders = []

        for i, article in enumerate(self.articles):
            try:
                logging.info(f"Extracting stakeholders from article {i+1}/{len(self.articles)}")
                article_content = article.get('content', '')
                article_metadata = f"Title: {article.get('title', 'Unknown')}\nDate: {article.get('date', 'Unknown')}\nMedia: {article.get('media_outlet', 'Unknown')}"

                chatbot = ChatGPT(
                    system_prompt=f"You are a political analyst expert in identifying stakeholders relevant to politicians.",
                    model_name="gpt-4o-mini",
                    temperature=0,
                    max_tokens=1500
                )

                stakeholder_prompt = f"""
Analyze the following article and identify all stakeholders (people and organizations) that are directly relevant to {self.politician_name}.

A stakeholder is considered relevant if they:
1. Have direct interaction with {self.politician_name}
2. Influence {self.politician_name}'s political decisions or career
3. Are influenced by {self.politician_name}'s actions or policies
4. Share political interests or opposition with {self.politician_name}
5. Exclude {self.politician_name} from the list.

For each identified stakeholder, provide:
- Name (full, official name of the person or organization)
- Type (either "person" or "organization")
- Description (brief explanation of who this stakeholder is)

Format your response as a JSON list of objects:
[
    {{
    "name": "stakeholder name",
    "type": "person/organization",
    "description": "brief description"
    }},
    ...
]

Article Metadata:
{article_metadata}

Article Content:
{article_content}

If no stakeholders relevant to {self.politician_name} are found in this article, return an empty list: []
"""

                response = chatbot.ask(stakeholder_prompt)
                print(response)

                # Process the response to extract stakeholders
                try:
                    # Clean the response to ensure valid JSON
                    json_start = response.find('[')
                    json_end = response.rfind(']') + 1

                    if json_start != -1 and json_end != -1:
                        cleaned_json = response[json_start:json_end]
                        article_stakeholders = json.loads(cleaned_json)

                        # Add article reference to each stakeholder
                        for stakeholder in article_stakeholders:
                            stakeholder['article_ref'] = {
                                'title': article.get('title', 'Unknown'),
                                'date': article.get('date', 'Unknown'),
                                'media_outlet': article.get('media_outlet', 'Unknown')
                            }
                            stakeholder['content'] = article.get('content', '')

                        all_stakeholders.extend(article_stakeholders)
                        logging.info(f"Extracted {len(article_stakeholders)} stakeholders from article {i+1}")
                    else:
                        logging.warning(f"No valid JSON found in response for article {i+1}")
                except json.JSONDecodeError:
                    logging.error(f"Failed to parse stakeholder JSON for article {i+1}")
                    continue
                
            except Exception as e:
                logging.error(f"Error extracting stakeholders from article {i+1}: {str(e)}")
                continue
            
        # Deduplicate stakeholders by name/type
        unique_stakeholders = {}
        for stakeholder in all_stakeholders:
            key = f"{stakeholder['name']}|{stakeholder['type']}"
            if key not in unique_stakeholders:
                unique_stakeholders[key] = {
                    'name': stakeholder['name'],
                    'type': stakeholder['type'],
                    'description': stakeholder['description'],
                    'article_refs': [stakeholder['article_ref']],
                    'contents': [stakeholder['content']]
                }
            else:
                unique_stakeholders[key]['article_refs'].append(stakeholder['article_ref'])
                unique_stakeholders[key]['contents'].append(stakeholder['content'])

        stakeholders_list = list(unique_stakeholders.values())
        print(stakeholders_list)
        logging.info(f"Identified {len(stakeholders_list)} unique stakeholders after deduplication")

        # Step 2: Enhance stakeholder information with relationship data
        enhanced_stakeholders = []

        for i, stakeholder in enumerate(stakeholders_list):
            try:
                logging.info(f"Enhancing stakeholder data for {stakeholder['name']} ({i+1}/{len(stakeholders_list)})")

                # Join article contents for context, but limit size
                combined_content = "\n\n".join(stakeholder['contents'])
                if len(combined_content) > 10000:  # Limit to ~10k characters
                    combined_content = combined_content[:10000] + "..."

                chatbot = ChatGPT(
                    system_prompt=f"You are a political analyst specializing in stakeholder relationships.",
                    model_name="gpt-4o-mini",
                    temperature=0,
                    max_tokens=800
                )

                relationship_prompt = f"""
Analyze the relationship between {self.politician_name} and the stakeholder {stakeholder['name']} ({stakeholder['type']}) based on the following article content.

Stakeholder description: {stakeholder['description']}

Extract the following information:
1. Role/Position: The stakeholder's professional role/position (specific title/role) or organizational type.
2. Relationship Type: How the stakeholder relates to {self.politician_name} (e.g., ally, opponent, mentor, colleague, constituent)
3. Influence Description: How this stakeholder influences or is influenced by {self.politician_name} in the political landscape

Format your response as a JSON object:
{{
    "role": "stakeholder's role or position",
    "relationship_type": "type of relationship with the politician",
    "influence_description": "description of political influence/relevance"
}}

Article content:
{combined_content}
"""

                response = chatbot.ask(relationship_prompt)
                print(response)

                # Process the response to extract relationship info
                try:
                    # Clean the response to ensure valid JSON
                    json_start = response.find('{')
                    json_end = response.rfind('}') + 1

                    if json_start != -1 and json_end != -1:
                        cleaned_json = response[json_start:json_end]
                        relationship_data = json.loads(cleaned_json)

                        # Merge relationship data with stakeholder info
                        enhanced_stakeholder = {
                            **stakeholder,
                            'role': relationship_data.get('role', 'Unknown'),
                            'relationship_type': relationship_data.get('relationship_type', 'Unknown'),
                            'influence_description': relationship_data.get('influence_description', ''),
                            'mentions': len(stakeholder['article_refs'])
                        }

                        # Remove verbose content data to keep output manageable
                        enhanced_stakeholder.pop('contents', None)

                        enhanced_stakeholders.append(enhanced_stakeholder)

                    else:
                        logging.warning(f"No valid JSON found in relationship response for {stakeholder['name']}")
                        # Include stakeholder with default relationship values
                        stakeholder['role'] = 'Unknown'
                        stakeholder['relationship_type'] = 'Unknown'
                        stakeholder['influence_description'] = ''
                        stakeholder['mentions'] = len(stakeholder['article_refs'])
                        stakeholder.pop('contents', None)
                        enhanced_stakeholders.append(stakeholder)

                except json.JSONDecodeError:
                    logging.error(f"Failed to parse relationship JSON for {stakeholder['name']}")
                    # Include stakeholder with default relationship values
                    stakeholder['role'] = 'Unknown'
                    stakeholder['relationship_type'] = 'Unknown'
                    stakeholder['influence_description'] = ''
                    stakeholder['mentions'] = len(stakeholder['article_refs'])
                    stakeholder.pop('contents', None)
                    enhanced_stakeholders.append(stakeholder)

            except Exception as e:
                logging.error(f"Error enhancing stakeholder data for {stakeholder['name']}: {str(e)}")
                # Include stakeholder with default relationship values
                stakeholder['role'] = 'Unknown'
                stakeholder['relationship_type'] = 'Unknown'
                stakeholder['influence_description'] = ''
                stakeholder['mentions'] = len(stakeholder['article_refs'])
                stakeholder.pop('contents', None)
                enhanced_stakeholders.append(stakeholder)

        # Step 3: Define stakeholder categories
        # Build a descriptive list of stakeholders for categorization
        stakeholder_descriptions = []
        for s in enhanced_stakeholders:
            desc = f"Name: {s['name']}, Type: {s['type']}, Role: {s['role']}, " \
                   f"Relationship: {s['relationship_type']}, Description: {s['description']}, " \
                   f"Influence: {s['influence_description']}"
            stakeholder_descriptions.append(desc)

        all_descriptions = "\n\n".join(stakeholder_descriptions)

        chatbot = ChatGPT(
            system_prompt=f"You are a political analyst specializing in stakeholder categorization and mapping.",
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=1500
        )

        categories_prompt = f"""
Based on the following list of stakeholders related to {self.politician_name}, create 6-9 distinct categories to classify them.

Each category should group stakeholders based on their relationship to {self.politician_name}, their sector, influence type, or other relevant factors.

For each category you define, provide:
1. A clear, specific category name
2. A detailed description of the category and its significance to {self.politician_name}
3. The types of stakeholders that would belong in this category
4. The political implications of this stakeholder group for {self.politician_name}

Format your response as a JSON list:
[
    {{
    "category": "Category Name",
    "description": "Detailed category description",
    "stakeholder_types": "Types of stakeholders in this category",
    "political_implications": "Political implications of this stakeholder group"
    }},
    ...
]

Stakeholder List:
{all_descriptions}
"""

        categories_response = chatbot.ask(categories_prompt)
        print(categories_response)

        # Process the response to extract categories
        try:
            # Clean the response to ensure valid JSON
            json_start = categories_response.find('[')
            json_end = categories_response.rfind(']') + 1

            if json_start != -1 and json_end != -1:
                cleaned_json = categories_response[json_start:json_end]
                stakeholder_categories = json.loads(cleaned_json)
                logging.info(f"Created {len(stakeholder_categories)} stakeholder categories")
            else:
                logging.warning("No valid category JSON found in response")
                stakeholder_categories = [
                    {"category": "Political Associates", "description": "Direct political connections", "stakeholder_types": "Politicians, party members", "political_implications": "Direct influence on political agenda"},
                    {"category": "Government Entities", "description": "Government-related organizations", "stakeholder_types": "Departments, agencies", "political_implications": "Execution of policies"},
                    {"category": "Media Relations", "description": "Media entities and personnel", "stakeholder_types": "Journalists, news outlets", "political_implications": "Public image management"},
                    {"category": "Civil Society", "description": "Non-governmental organizations", "stakeholder_types": "NGOs, advocacy groups", "political_implications": "Public support mobilization"},
                    {"category": "Constituents", "description": "The public and voters", "stakeholder_types": "Voters, community members", "political_implications": "Electoral support"},
                    {"category": "Business Connections", "description": "Private sector entities", "stakeholder_types": "Companies, business leaders", "political_implications": "Economic policy influence"}
                ]
        except json.JSONDecodeError:
            logging.error("Failed to parse category JSON")
            stakeholder_categories = [
                {"category": "Political Associates", "description": "Direct political connections", "stakeholder_types": "Politicians, party members", "political_implications": "Direct influence on political agenda"},
                {"category": "Government Entities", "description": "Government-related organizations", "stakeholder_types": "Departments, agencies", "political_implications": "Execution of policies"},
                {"category": "Media Relations", "description": "Media entities and personnel", "stakeholder_types": "Journalists, news outlets", "political_implications": "Public image management"},
                {"category": "Civil Society", "description": "Non-governmental organizations", "stakeholder_types": "NGOs, advocacy groups", "political_implications": "Public support mobilization"},
                {"category": "Constituents", "description": "The public and voters", "stakeholder_types": "Voters, community members", "political_implications": "Electoral support"},
                {"category": "Business Connections", "description": "Private sector entities", "stakeholder_types": "Companies, business leaders", "political_implications": "Economic policy influence"}
            ]

        # Step 4: Classify each stakeholder into one of the categories
        categorized_stakeholders = []

        for s in enhanced_stakeholders:
            category_names = [cat["category"] for cat in stakeholder_categories]
            category_descriptions = "\n".join([f"{cat['category']}: {cat['description']}" for cat in stakeholder_categories])

            chatbot = ChatGPT(
                model_name="gpt-4o-mini",
                temperature=0,
                max_tokens=200
            )

            classify_prompt = f"""
Classify the following stakeholder into exactly one of these categories:
{category_descriptions}

Stakeholder:
- Name: {s['name']}
- Type: {s['type']}
- Role: {s['role']}
- Relationship with {self.politician_name}: {s['relationship_type']}
- Description: {s['description']}
- Influence: {s['influence_description']}

Respond with only the exact category name and nothing else. Choose from: {', '.join(category_names)}
    """

            try:
                category_response = chatbot.ask(classify_prompt).strip()
                print(category_response)

                # Find closest matching category
                if category_response in category_names:
                    matched_category = category_response
                else:
                    # Find closest match if exact match not found
                    matched_category = category_names[0]  # Default to first category
                    for cat_name in category_names:
                        if cat_name.lower() in category_response.lower():
                            matched_category = cat_name
                            break
                        
                # Add category to stakeholder
                s['category'] = matched_category
                categorized_stakeholders.append(s)

            except Exception as e:
                logging.error(f"Error classifying stakeholder {s['name']}: {str(e)}")
                s['category'] = "Uncategorized"
                categorized_stakeholders.append(s)

        # Step 5: Perform stakeholder mapping analysis for each category
        category_analyses = {}

        for category in stakeholder_categories:
            category_name = category["category"]
            stakeholders_in_category = [s for s in categorized_stakeholders if s['category'] == category_name]

            if not stakeholders_in_category:
                category_analyses[category_name] = {
                    "category_info": category,
                    "stakeholders": [],
                    "analysis": "No stakeholders identified in this category."
                }
                continue
            
            # Format stakeholder data for analysis
            stakeholder_data = []
            for s in stakeholders_in_category:
                stakeholder_data.append({
                    "name": s['name'],
                    "type": s['type'],
                    "role": s['role'],
                    "relationship_type": s['relationship_type'],
                    "mentions": s['mentions'],
                    "description": s['description'],
                    "influence_description": s['influence_description']
                })

            chatbot = ChatGPT(
                model_name="chatgpt-4o-latest",
                temperature=0,
                max_tokens=2000
            )

            analysis_prompt = f"""
Perform a comprehensive stakeholder mapping analysis for the following group of stakeholders related to {self.politician_name}.

These stakeholders belong to the category: "{category_name}"
Category description: {category["description"]}

Stakeholders in this category:
{json.dumps(stakeholder_data, indent=2)}

Your analysis should include:

1. Power-Interest Analysis:
    - Identify which stakeholders have high/low power and high/low interest regarding {self.politician_name}.
    - Explain the basis for these power and interest assessments.
    - Use these columns to format this part of the assessment: | Stakeholder | Power | Interest | Basis for Assessment |

2. Combined Network and Influence Analysis:
    - Provide an integrated analysis that merges network mapping with a detailed influence assessment.
    - Identify key network connectors and central influencers as well as any isolated or peripheral stakeholders, highlighting how these relationships could lead to alliances or conflicts.
    - Describe the communication flows and information exchange dynamics within the network.
    - Clearly pinpoint specific stakeholders capable of exerting direct influence on {self.politician_name}, detailing the nature of their influence (supportive, adversarial, or conditional) and explaining the factors and conditions that might shift their stance.
    - Format this section using a combination of narrative paragraphs and concise, descriptive bullet points that deliver rich information without excessive listing.

Format your response as a professionally written, comprehensive analysis with clear sections and insights. Start at the '###' header level as your output will be embedded in a markdown document. Use a mixture of bullet points and paragraphs to format your output. Do not use '---' symbols as delimiters.

Your output should only be the following two sections and nothing more: ###Power-Interest Analysis, ###Combined Network and Influence Analysis. No introduction or conclusion needed. Do not even include any concluding statement, stop straight after the last point in the Combined Network and Influence Analysis section.
"""

            analysis_response = chatbot.ask(analysis_prompt)
            print(analysis_response)

            category_analyses[category_name] = {
                "category_info": category,
                "stakeholders": stakeholders_in_category,
                "analysis": analysis_response
            }

        # Step 6: Create the final result structure
        result = {
            "stakeholders": categorized_stakeholders,
            "categories": stakeholder_categories,
            "category_analyses": category_analyses
        }

        # Save the results
        with open(stakeholder_path, 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=2)

        # Generate markdown report
        self._generate_stakeholder_analysis_report(result)

        return result

    def _generate_stakeholder_analysis_report(self, stakeholder_data):
        """
        Generate a comprehensive markdown report of stakeholder analysis.
        """
        stakeholders = stakeholder_data["stakeholders"]
        categories = stakeholder_data["categories"]
        category_analyses = stakeholder_data["category_analyses"]

        # Setup output path
        output_path = os.path.join(
            self.general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders",
            f"StakeholderAnalysisReport_{self.politician_name.replace(' ', '_')}.md"
        )

        # Generate visualization data for stakeholder categories
        category_counts = Counter([s['category'] for s in stakeholders])

        # Generate person vs organization counts
        type_counts = Counter([s['type'] for s in stakeholders])

        # Generate relationship type distribution
        relationship_counts = Counter([s['relationship_type'] for s in stakeholders])

        # Generate stakeholder visualization charts
        charts_folder = os.path.join(self.general_folder, "Outputs", "Visualizations")
        os.makedirs(charts_folder, exist_ok=True)

        # Create category distribution chart
        try:
            category_data = pd.Series(category_counts)
            category_chart_path = os.path.join(charts_folder, f"StakeholderCategories_{self.politician_name.replace(' ', '_')}.png")

            plt.figure(figsize=(10, 7))
            category_data.plot(kind='bar', color='skyblue')
            plt.title(f'Stakeholder Categories - {self.politician_name}')
            plt.ylabel('Number of Stakeholders')
            plt.xlabel('Category')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(category_chart_path)
            plt.close()

            category_chart_rel_path = os.path.relpath(
                category_chart_path, 
                os.path.dirname(output_path)
            )
        except Exception as e:
            logging.error(f"Error creating category chart: {str(e)}")
            category_chart_rel_path = None

        # Create type distribution chart (person vs organization)
        try:
            type_data = pd.Series(type_counts)
            type_chart_path = os.path.join(charts_folder, f"StakeholderTypes_{self.politician_name.replace(' ', '_')}.png")

            plt.figure(figsize=(8, 6))
            type_data.plot(kind='pie', autopct='%1.1f%%')
            plt.title(f'Stakeholder Types - {self.politician_name}')
            plt.ylabel('')
            plt.tight_layout()
            plt.savefig(type_chart_path)
            plt.close()

            type_chart_rel_path = os.path.relpath(
                type_chart_path, 
                os.path.dirname(output_path)
            )
        except Exception as e:
            logging.error(f"Error creating type chart: {str(e)}")
            type_chart_rel_path = None

        # Start building the markdown report
        md_content = f"""# Stakeholder Analysis for {self.politician_name}

## Overview

This report provides a comprehensive analysis of stakeholders related to {self.politician_name} based on media coverage. The analysis identifies key individuals and organizations, categorizes them based on their relationship with {self.politician_name}, and provides strategic insights for each stakeholder group.

## Summary Statistics

- **Total Stakeholders Identified**: {len(stakeholders)}
- **Organizations**: {type_counts.get('organization', 0)}
- **Individuals**: {type_counts.get('person', 0)}
- **Stakeholder Categories**: {len(categories)}

## Stakeholder Distribution by Category

"""

        if category_chart_rel_path:
            md_content += f"![Stakeholder Categories]({category_chart_rel_path})\n\n"

        # Add category analyses
        md_content += """
## Category Analysis

This section provides a detailed analysis of each stakeholder category, including power-interest mapping, network analysis, and political impact assessment.

"""

        for category_name, analysis in category_analyses.items():
            category_info = analysis["category_info"]
            stakeholders_in_category = analysis["stakeholders"]
            analysis_text = analysis["analysis"]

            md_content += f"### {category_name}\n\n"
            md_content += f"{category_info['description']}"
            md_content += f"{category_info['political_implications']}\n\n"
            md_content += f"{analysis_text}\n\n"


            md_content += "\n\n---\n\n"

        # Generate the appendix content for the complete stakeholder list
        appendix_content = f"""# Stakeholder Appendix for {self.politician_name}

        This appendix contains a complete list of all identified stakeholders.

        | Name | Type | Category | Role | Relationship | Mentions |
        |------|------|----------|------|-------------|----------|
        """

        for s in sorted(stakeholders, key=lambda x: x['name']):
            appendix_content += f"| {s['name']} | {s['type']} | {s['category']} | {s['role']} | {s['relationship_type']} | {s['mentions']} |\n"

        # Define a separate output path for the appendix
        appendix_path = os.path.join(
            self.general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders",
            f"StakeholderAnalysisAppendix_{self.politician_name.replace(' ', '_')}.md"
        )

        # Save the appendix document
        with open(appendix_path, 'w', encoding='utf-8') as f:
            f.write(appendix_content)

        logging.info(f"Stakeholder appendix report saved to {appendix_path}")

        # Save the markdown report
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(md_content)

        logging.info(f"Stakeholder analysis report saved to {output_path}")
    
    def create_timeline_analysis(self):
        """
        Create a chronological analysis of the politician's media coverage.
        Identifies key events and shifts in narrative.
        Returns timeline data suitable for visualization.
        """
        logging.info("Beginning timeline analysis")
        
        # Ensure articles are sorted chronologically
        articles = sorted(self.articles, key=lambda x: x.get('timestamp', 0))
        
        # Group articles by month
        timeline_data = defaultdict(list)
        monthly_stats = {}
        
        for article in articles:
            # Extract date and format as YYYY-MM
            try:
                date_obj = datetime.fromtimestamp(article.get('timestamp', 0))
                month_key = date_obj.strftime('%Y-%m')
            except (ValueError, TypeError, OSError):
                # If timestamp is invalid, try parsing the date string
                try:
                    date_obj = datetime.strptime(article.get('date', ''), '%B %d, %Y')
                    month_key = date_obj.strftime('%Y-%m')
                except (ValueError, TypeError):
                    # If all else fails, use a default
                    month_key = 'unknown'
            
            timeline_data[month_key].append(article)
        
        # Calculate monthly statistics
        for month, month_articles in timeline_data.items():
            sentiment_scores = [a.get('sentiment_score', 0) for a in month_articles]
            tones = Counter(a.get('tone', 'Neutral') for a in month_articles)
            
            monthly_stats[month] = {
                'count': len(month_articles),
                'avg_sentiment': sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0,
                'tone_distribution': {
                    'Positive': tones.get('Positive', 0),
                    'Neutral': tones.get('Neutral', 0),
                    'Negative': tones.get('Negative', 0)
                },
                'narrative_distribution': Counter(
                    a.get('narrative_category', 'Unknown') for a in month_articles
                ),
                'top_outlets': Counter(
                    a.get('media_outlet', 'Unknown') for a in month_articles
                ).most_common(3)
            }
        
        # Identify significant shifts in narrative or sentiment
        sorted_months = sorted(monthly_stats.keys())
        shifts = []
        
        for i in range(1, len(sorted_months)):
            prev_month = sorted_months[i-1]
            curr_month = sorted_months[i]
            
            # Check for sentiment shifts
            sentiment_diff = monthly_stats[curr_month]['avg_sentiment'] - monthly_stats[prev_month]['avg_sentiment']
            if abs(sentiment_diff) >= 1.5:  # Significant shift threshold
                shifts.append({
                    'type': 'sentiment_shift',
                    'from_month': prev_month,
                    'to_month': curr_month,
                    'magnitude': sentiment_diff,
                    'from_value': monthly_stats[prev_month]['avg_sentiment'],
                    'to_value': monthly_stats[curr_month]['avg_sentiment']
                })
            
            # Check for volume shifts
            volume_ratio = monthly_stats[curr_month]['count'] / monthly_stats[prev_month]['count'] if monthly_stats[prev_month]['count'] > 0 else float('inf')
            if volume_ratio >= 2 or volume_ratio <= 0.5:
                shifts.append({
                    'type': 'volume_shift',
                    'from_month': prev_month,
                    'to_month': curr_month,
                    'magnitude': volume_ratio,
                    'from_value': monthly_stats[prev_month]['count'],
                    'to_value': monthly_stats[curr_month]['count']
                })
        
        # Identify key events for each month (significant articles)
        key_events = {}
        
        for month, month_articles in timeline_data.items():
            # Sort articles by absolute sentiment score (most polarized first)
            sorted_articles = sorted(
                month_articles, 
                key=lambda x: abs(x.get('sentiment_score', 0)),
                reverse=True
            )
            
            # Take up to 3 most significant articles
            significant_articles = sorted_articles[:min(3, len(sorted_articles))]
            
            key_events[month] = [
                {
                    'title': article.get('title', 'Untitled'),
                    'date': article.get('date', 'Unknown'),
                    'media_outlet': article.get('media_outlet', 'Unknown'),
                    'sentiment_score': article.get('sentiment_score', 0),
                    'tone': article.get('tone', 'Neutral'),
                    'narrative_category': article.get('narrative_category', 'Unknown')
                }
                for article in significant_articles
            ]
        
        # Combine all timeline data
        timeline_result = {
            'monthly_stats': monthly_stats,
            'shifts': shifts,
            'key_events': key_events,
            'timeline_data': {month: len(articles) for month, articles in timeline_data.items()}
        }
        
        # Save timeline data
        timeline_path = os.path.join(
            self.general_folder, "Outputs", "PoliticianAnalysis", "Timeline",
            f"Timeline_{self.politician_name.replace(' ', '_')}.json"
        )
        
        with open(timeline_path, 'w', encoding='utf-8') as f:
            json.dump(timeline_result, f, indent=2)
        
        return timeline_result
    
    def generate_network_graph(self):
        """
        Generate a network graph of the politician's relationships with stakeholders.
        Returns data suitable for visualization.
        """
        # Get stakeholder data
        stakeholders = self.analyze_stakeholders()
        
        # Create graph
        G = nx.Graph()
        
        # Add politician as central node
        G.add_node(self.politician_name, type='politician', size=20)
        
        # Add stakeholders as nodes
        for stakeholder in stakeholders:
            if stakeholder['mentions'] >= 2:  # Only include stakeholders mentioned at least twice
                G.add_node(
                    stakeholder['name'], 
                    type=stakeholder['type'],
                    size=5 + stakeholder['mentions'],
                    sentiment=stakeholder['sentiment_score']
                )
                
                # Add edge between politician and stakeholder
                G.add_edge(
                    self.politician_name, 
                    stakeholder['name'], 
                    weight=stakeholder['mentions'],
                    relationship=stakeholder['relationship']
                )
        
        # Find connections between stakeholders
        for i, stakeholder1 in enumerate(stakeholders):
            if stakeholder1['mentions'] < 2:
                continue
                
            for j, stakeholder2 in enumerate(stakeholders[i+1:], i+1):
                if stakeholder2['mentions'] < 2:
                    continue
                    
                # Check if stakeholders are mentioned in the same articles
                s1_articles = set(context['article_date'] + context['media_outlet'] 
                              for context in stakeholder1['contexts'])
                s2_articles = set(context['article_date'] + context['media_outlet'] 
                              for context in stakeholder2['contexts'])
                
                # If they appear in at least 2 same articles, add edge
                common_articles = s1_articles.intersection(s2_articles)
                if len(common_articles) >= 2:
                    G.add_edge(
                        stakeholder1['name'],
                        stakeholder2['name'],
                        weight=len(common_articles)
                    )
        
        # Convert to JSON for visualization
        network_data = {
            'nodes': [
                {
                    'id': node,
                    'type': G.nodes[node].get('type', 'unknown'),
                    'size': G.nodes[node].get('size', 5),
                    'sentiment': G.nodes[node].get('sentiment', 0)
                }
                for node in G.nodes()
            ],
            'links': [
                {
                    'source': source,
                    'target': target,
                    'weight': G.edges[source, target].get('weight', 1),
                    'relationship': G.edges[source, target].get('relationship', '')
                }
                for source, target in G.edges()
            ]
        }
        
        # Save network data
        network_path = os.path.join(
            self.general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders",
            f"NetworkGraph_{self.politician_name.replace(' ', '_')}.json"
        )
        
        with open(network_path, 'w', encoding='utf-8') as f:
            json.dump(network_data, f, indent=2)
        
        return network_data
    
    def generate_complete_analysis(self):
        """
        Generate complete political profile analysis.
        Returns markdown content with comprehensive analysis.
        """
        logging.info(f"Generating complete analysis for {self.politician_name}")
        
        # Ensure all analyses are completed
        self.preprocess_articles()
        narrative_categories = self.analyze_narratives()
        policy_areas = self.analyze_policy_positions()
        stakeholders = self.analyze_stakeholders()
        timeline_data = self.create_timeline_analysis()
        network_data = self.generate_network_graph()
        
        # Create visualizations
        self._create_visualizations(narrative_categories, policy_areas, stakeholders, timeline_data)
        
        # Generate executive summary
        summary = self._generate_executive_summary()
        
        # Generate the markdown document
        markdown_content = f"""
# Political Profile Analysis: {self.politician_name}

{summary}

## Table of Contents
- [Media Coverage Overview](#media-coverage-overview)
- [Narrative Analysis](#narrative-analysis)
- [Policy Position Analysis](#policy-position-analysis)
- [Timeline Analysis](#timeline-analysis)
- [Stakeholder Analysis](#stakeholder-analysis)

## Media Coverage Overview

{self._generate_media_coverage_overview()}

## Narrative Analysis

{self._generate_narrative_analysis(narrative_categories)}

## Policy Position Analysis

{self._generate_policy_analysis(policy_areas)}

## Timeline Analysis

{self._generate_timeline_analysis(timeline_data)}

## Stakeholder Analysis

{self._generate_stakeholder_analysis(stakeholders, network_data)}

---

Generated on {datetime.now().strftime('%Y-%m-%d')}
        """
        
        # Apply translations if needed
        if self.language.lower() != 'english':
            markdown_content = translate_content(markdown_content, 'auto', self.language)
        
        # Process citations
        processor = CitationProcessor()
        markdown_content = processor.process_citations_in_markdown(markdown_content)
        
        # Save complete analysis
        output_path = os.path.join(
            self.general_folder, "Outputs", "CompiledOutputs", 
            f"PoliticalProfile_{self.politician_name.replace(' ', '_')}.md"
        )
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        
        return markdown_content
    
    def _create_visualizations(self, narrative_categories, policy_areas, stakeholders, timeline_data):
        """Create visualizations for the analysis."""
        
        # Prepare directory
        viz_dir = os.path.join(self.general_folder, "Outputs", "Visualizations")
        os.makedirs(viz_dir, exist_ok=True)
        
        # 1. Narrative Distribution Pie Chart
        narrative_counts = Counter(article.get('narrative_category', 'Unknown') for article in self.articles)
        narrative_series = pd.Series(narrative_counts)
        
        fig = create_professional_pie(
            narrative_series,
            f'Distribution of Narratives about {self.politician_name}',
            figsize=(10, 8)
        )
        
        narrative_chart_path = os.path.join(viz_dir, f"Narrative_Distribution_{self.politician_name.replace(' ', '_')}.png")
        plt.savefig(narrative_chart_path, bbox_inches='tight', dpi=300)
        plt.close(fig)
        
        # 2. Policy Areas Sentiment Bar Chart
        if policy_areas:
            policy_names = [policy['policy_area'] for policy in policy_areas]
            sentiment_values = [policy['sentiment']['average'] for policy in policy_areas]
            
            fig, ax = plt.subplots(figsize=(12, 8))
            bars = ax.bar(policy_names, sentiment_values)
            
            # Color bars based on sentiment
            for i, bar in enumerate(bars):
                if sentiment_values[i] > 0.2:
                    bar.set_color('#4CAF50')  # Green for positive
                elif sentiment_values[i] < -0.2:
                    bar.set_color('#F44336')  # Red for negative
                else:
                    bar.set_color('#2196F3')  # Blue for neutral
            
            ax.set_title(f'Sentiment Analysis by Policy Area - {self.politician_name}', fontsize=16)
            ax.set_xlabel('Policy Area', fontsize=12)
            ax.set_ylabel('Average Sentiment (-1 to 1)', fontsize=12)
            ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
            ax.set_ylim(-1.2, 1.2)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            policy_chart_path = os.path.join(viz_dir, f"Policy_Sentiment_{self.politician_name.replace(' ', '_')}.png")
            plt.savefig(policy_chart_path, bbox_inches='tight', dpi=300)
            plt.close(fig)
        
        # 3. Timeline Chart
        if timeline_data and 'timeline_data' in timeline_data:
            months = sorted(timeline_data['timeline_data'].keys())
            counts = [timeline_data['timeline_data'][month] for month in months]
            
            # Format month labels
            month_labels = []
            for month in months:
                try:
                    date_obj = datetime.strptime(month, '%Y-%m')
                    month_labels.append(date_obj.strftime('%b %Y'))
                except ValueError:
                    month_labels.append(month)
            
            fig, ax = plt.subplots(figsize=(14, 8))
            ax.plot(month_labels, counts, marker='o', linewidth=2, markersize=8, color='#1976D2')
            
            # Add sentiment overlay if available
            if 'monthly_stats' in timeline_data:
                sentiment_values = []
                for month in months:
                    if month in timeline_data['monthly_stats']:
                        sentiment_values.append(timeline_data['monthly_stats'][month].get('avg_sentiment', 0))
                    else:
                        sentiment_values.append(0)
                
                # Create secondary y-axis for sentiment
                ax2 = ax.twinx()
                ax2.plot(month_labels, sentiment_values, marker='s', linewidth=2, 
                       linestyle='--', color='#FF5722', alpha=0.7, label='Average Sentiment')
                ax2.set_ylabel('Average Sentiment (-5 to 5)', fontsize=12, color='#FF5722')
                ax2.set_ylim(-5, 5)
                ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
            
            ax.set_title(f'Media Coverage Timeline - {self.politician_name}', fontsize=16)
            ax.set_xlabel('Month', fontsize=12)
            ax.set_ylabel('Number of Articles', fontsize=12)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            timeline_chart_path = os.path.join(viz_dir, f"Coverage_Timeline_{self.politician_name.replace(' ', '_')}.png")
            plt.savefig(timeline_chart_path, bbox_inches='tight', dpi=300)
            plt.close(fig)
        
        # 4. Top Stakeholders Bar Chart
        if stakeholders:
            # Get top 10 stakeholders by mentions
            top_stakeholders = sorted(stakeholders, key=lambda x: x['mentions'], reverse=True)[:10]
            names = [s['name'] for s in top_stakeholders]
            mentions = [s['mentions'] for s in top_stakeholders]
            
            fig, ax = plt.subplots(figsize=(12, 8))
            bars = ax.barh(names[::-1], mentions[::-1])  # Reverse order for better visualization
            
            # Color bars based on stakeholder type
            for i, bar in enumerate(bars):
                if top_stakeholders[len(top_stakeholders)-i-1]['type'] == 'person':
                    bar.set_color('#2196F3')  # Blue for people
                else:
                    bar.set_color('#FF9800')  # Orange for organizations
            
            ax.set_title(f'Top Stakeholders in Media Coverage - {self.politician_name}', fontsize=16)
            ax.set_xlabel('Number of Mentions', fontsize=12)
            plt.tight_layout()
            
            stakeholder_chart_path = os.path.join(viz_dir, f"Top_Stakeholders_{self.politician_name.replace(' ', '_')}.png")
            plt.savefig(stakeholder_chart_path, bbox_inches='tight', dpi=300)
            plt.close(fig)
    
    def _generate_executive_summary(self):
        """Generate executive summary of the political profile."""
        
        # Compile data for summary
        total_articles = len(self.articles)
        date_range = "Unknown"
        
        # Try to determine date range
        dates = sorted([
            datetime.fromtimestamp(article.get('timestamp', 0)) 
            for article in self.articles 
            if article.get('timestamp', 0) > 0
        ])
        
        if dates:
            start_date = dates[0].strftime('%B %d, %Y')
            end_date = dates[-1].strftime('%B %d, %Y')
            date_range = f"{start_date} to {end_date}"
        
        # Calculate overall sentiment
        sentiment_scores = [article.get('sentiment_score', 0) for article in self.articles]
        avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0
        
        # Get tone distribution
        tone_distribution = Counter(article.get('tone', 'Neutral') for article in self.articles)
        
        # Generate summary using AI
        chatbot = ChatGPT(
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=1500
        )
        
        summary_prompt = f"""
Generate an executive summary for a political profile analysis of {self.politician_name}.

Key statistics:
- Total articles analyzed: {total_articles}
- Coverage period: {date_range}
- Average sentiment: {avg_sentiment:.2f} (scale: -5 to +5)
- Tone distribution: Positive ({tone_distribution.get('Positive', 0)}), Neutral ({tone_distribution.get('Neutral', 0)}), Negative ({tone_distribution.get('Negative', 0)})

Structure the summary with the following sections:
1. Overview: A brief introduction that summarizes the scope of the analysis and key findings.
2. Main Narratives: The key narrative frames used to portray this politician.
3. Policy Focus: Main policy areas discussed in media coverage.
4. Sentiment Patterns: Key insights about sentiment and tone in the coverage.
5. Key Relationships: Significant stakeholders and relationships portrayed in media.

Make the summary concise but comprehensive, highlighting the most significant patterns and insights from the data.
Include a title at the beginning: "## Executive Summary"
"""
        
        summary = chatbot.ask(summary_prompt)
        return summary
    
    def _generate_media_coverage_overview(self):
        """Generate media coverage overview section."""
        
        # Calculate media outlet distribution
        outlet_counts = Counter(article.get('media_outlet', 'Unknown') for article in self.articles)
        top_outlets = outlet_counts.most_common(10)
        
        # Format outlet distribution as markdown
        outlet_md = "### Media Outlet Distribution\n\n"
        outlet_md += "| Media Outlet | Number of Articles | Percentage |\n"
        outlet_md += "|-------------|---------------------|------------|\n"
        
        total_articles = len(self.articles)
        for outlet, count in top_outlets:
            percentage = (count / total_articles) * 100
            outlet_md += f"| {outlet} | {count} | {percentage:.1f}% |\n"
        
        # Add visualization reference
        outlet_md += "\n![Media Outlet Distribution](../Visualizations/"
        outlet_md += f"Narrative_Distribution_{self.politician_name.replace(' ', '_')}.png)\n\n"
        
        # Generate AI analysis
        chatbot = ChatGPT(
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=1000
        )
        
        overview_prompt = f"""
Analyze the media coverage of {self.politician_name} based on the following data:

Media Outlet Distribution:
{', '.join([f"{outlet}: {count}" for outlet, count in top_outlets])}

Total Articles: {len(self.articles)}

Provide an analysis of:
1. The breadth and diversity of media coverage
2. Any notable patterns in which outlets cover the politician most frequently
3. Potential implications of this coverage distribution

Keep your analysis to 2-3 paragraphs and focus on substantive insights rather than just restating the data.
"""
        
        coverage_analysis = chatbot.ask(overview_prompt)
        
        # Combine everything
        media_overview = f"""
### Overview of Media Coverage

{coverage_analysis}

{outlet_md}
"""
        
        return media_overview
    
    def _generate_narrative_analysis(self, narrative_categories):
        """Generate narrative analysis section."""
        
        if not narrative_categories:
            return "No narrative categories identified."
        
        narrative_md = "### Primary Narrative Frames\n\n"
        narrative_md += "![Narrative Distribution](../Visualizations/"
        narrative_md += f"Narrative_Distribution_{self.politician_name.replace(' ', '_')}.png)\n\n"
        
        # Analyze each narrative category
        for category in narrative_categories:
            if not category.get('articles'):
                continue
                
            category_name = category['category']
            category_desc = category['description']
            article_count = len(category['articles'])
            
            narrative_md += f"#### {category_name}\n\n"
            narrative_md += f"*{category_desc}*\n\n"
            
            # Compile examples for this category
            examples = []
            for article in category['articles'][:3]:  # Take up to 3 examples
                examples.append({
                    'title': article.get('title', 'Untitled'),
                    'outlet': article.get('media_outlet', 'Unknown'),
                    'date': article.get('date', 'Unknown'),
                    'content': article.get('content', '')[:500] + "..."  # Truncate for analysis
                })
            
            # Generate AI analysis of this narrative category
            chatbot = ChatGPT(
                model_name="chatgpt-4o-latest",
                temperature=0,
                max_tokens=1000
            )
            
            category_prompt = f"""
Analyze the "{category_name}" narrative frame used to portray {self.politician_name} in media coverage.

Number of articles in this category: {article_count}
Category description: {category_desc}

Examples of articles in this category:
{json.dumps(examples, indent=2)}

Provide an analysis that:
1. Explains how this narrative frame portrays the politician
2. Identifies key language patterns or themes within this frame
3. Discusses any potential implications of this framing
4. Cites specific examples from the article excerpts provided

Write your analysis in 2-3 paragraphs. Focus on concrete analysis rather than generalizations.
"""
            
            category_analysis = chatbot.ask(category_prompt)
            narrative_md += f"{category_analysis}\n\n"
            
            # Add a few example article references
            narrative_md += "**Example articles in this category:**\n\n"
            for article in category['articles'][:5]:  # Take up to 5 examples
                title = article.get('title', 'Untitled')
                outlet = article.get('media_outlet', 'Unknown')
                date = article.get('date', 'Unknown')
                
                narrative_md += f"- *{title}* ({outlet}, {date})\n"
            
            narrative_md += "\n---\n\n"
        
        return narrative_md
    
    def _generate_policy_analysis(self, policy_areas):
        """Generate policy position analysis section."""
        
        if not policy_areas:
            return "No policy areas identified."
        
        policy_md = "### Policy Positions in Media Coverage\n\n"
        policy_md += "![Policy Sentiment](../Visualizations/"
        policy_md += f"Policy_Sentiment_{self.politician_name.replace(' ', '_')}.png)\n\n"
        
        # Sort policy areas by coverage volume
        sorted_policies = sorted(
            policy_areas,
            key=lambda x: len(x.get('positions', [])),
            reverse=True
        )
        
        for policy in sorted_policies:
            policy_name = policy['policy_area']
            policy_desc = policy['description']
            positions_count = len(policy['positions'])
            
            if positions_count == 0:
                continue
            
            policy_md += f"#### {policy_name}\n\n"
            policy_md += f"*{policy_desc}*\n\n"
            
            # Calculate sentiment metrics
            sentiment_data = policy['sentiment']
            sentiment_text = "Neutral"
            if sentiment_data['average'] > 0.3:
                sentiment_text = "Predominantly Positive"
            elif sentiment_data['average'] > 0:
                sentiment_text = "Slightly Positive"
            elif sentiment_data['average'] < -0.3:
                sentiment_text = "Predominantly Negative"
            elif sentiment_data['average'] < 0:
                sentiment_text = "Slightly Negative"
            
            policy_md += f"**Media Sentiment:** {sentiment_text} ({sentiment_data['average']:.2f})\n\n"
            
            # Generate AI analysis of this policy area
            chatbot = ChatGPT(
                model_name="chatgpt-4o-latest",
                temperature=0,
                max_tokens=1200
            )
            
            policy_prompt = f"""
Analyze how {self.politician_name}'s positions on {policy_name} are portrayed in media coverage.

Policy description: {policy_desc}
Media sentiment: {sentiment_text} ({sentiment_data['average']:.2f})

Specific positions and statements reported in media:
{chr(10).join(policy['positions'])}

Provide an analysis that:
1. Summarizes how the politician's positions on this issue are characterized
2. Identifies consistency or evolution in their reported stance
3. Notes how media framing might affect public perception of these positions
4. Cites specific examples from the reported positions

Write your analysis in 2-3 paragraphs. Be specific and substantive rather than general.
"""
            
            policy_analysis = chatbot.ask(policy_prompt)
            policy_md += f"{policy_analysis}\n\n"
            
            policy_md += "**Reported Positions:**\n\n"
            for position in policy['positions'][:5]:  # List up to 5 positions
                policy_md += f"- {position}\n"
            
            if len(policy['positions']) > 5:
                policy_md += f"\n*{len(policy['positions']) - 5} more positions reported...*\n"
            
            policy_md += "\n---\n\n"
        
        return policy_md
    
    def _generate_timeline_analysis(self, timeline_data):
        """Generate timeline analysis section."""
        
        if not timeline_data:
            return "No timeline data available."
        
        timeline_md = "### Coverage Timeline\n\n"
        timeline_md += "![Coverage Timeline](../Visualizations/"
        timeline_md += f"Coverage_Timeline_{self.politician_name.replace(' ', '_')}.png)\n\n"
        
        # Add significant shifts section
        shifts = timeline_data.get('shifts', [])
        if shifts:
            timeline_md += "#### Key Shifts in Coverage\n\n"
            
            for shift in shifts:
                shift_type = shift['type']
                from_month = shift['from_month']
                to_month = shift['to_month']
                
                try:
                    from_date = datetime.strptime(from_month, '%Y-%m').strftime('%B %Y')
                    to_date = datetime.strptime(to_month, '%Y-%m').strftime('%B %Y')
                except ValueError:
                    from_date = from_month
                    to_date = to_month
                
                if shift_type == 'sentiment_shift':
                    direction = "more positive" if shift['magnitude'] > 0 else "more negative"
                    timeline_md += f"- **Sentiment shift ({direction})** from {from_date} to {to_date}\n"
                    timeline_md += f"  - From {shift['from_value']:.2f} to {shift['to_value']:.2f}\n"
                
                elif shift_type == 'volume_shift':
                    direction = "increase" if shift['magnitude'] > 1 else "decrease"
                    timeline_md += f"- **Coverage volume {direction}** from {from_date} to {to_date}\n"
                    timeline_md += f"  - From {shift['from_value']} to {shift['to_value']} articles\n"
            
            timeline_md += "\n"
        
        # Add key events section
        key_events = timeline_data.get('key_events', {})
        if key_events:
            timeline_md += "#### Key Events Timeline\n\n"
            
            # Sort months chronologically
            sorted_months = sorted(key_events.keys())
            
            for month in sorted_months:
                events = key_events[month]
                if not events:
                    continue
                
                try:
                    month_display = datetime.strptime(month, '%Y-%m').strftime('%B %Y')
                except ValueError:
                    month_display = month
                
                timeline_md += f"**{month_display}**\n\n"
                
                for event in events:
                    title = event.get('title', 'Untitled')
                    outlet = event.get('media_outlet', 'Unknown')
                    date = event.get('date', 'Unknown')
                    narrative = event.get('narrative_category', 'Unknown')
                    
                    timeline_md += f"- *{title}* ({outlet}, {date})\n"
                    timeline_md += f"  - Narrative: {narrative}\n"
                
                timeline_md += "\n"
        
        # Generate AI analysis of timeline trends
        chatbot = ChatGPT(
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=1500
        )
        
        timeline_prompt = f"""
Analyze the timeline of media coverage for {self.politician_name} based on the following data:

Monthly statistics:
{json.dumps(timeline_data.get('monthly_stats', {}), indent=2)}

Key shifts in coverage:
{json.dumps(shifts, indent=2)}

Provide an analysis that:
1. Identifies patterns in how coverage has evolved over time
2. Explains significant shifts in sentiment or volume
3. Highlights potential correlations between real-world events and media coverage
4. Discusses the implications of these temporal patterns

Write your analysis in 3-4 paragraphs. Focus on identifying meaningful trends and explaining their significance.
"""
        
        timeline_analysis = chatbot.ask(timeline_prompt)
        timeline_md += f"\n#### Analysis of Coverage Trends\n\n{timeline_analysis}\n\n"
        
        return timeline_md
    
    def _generate_stakeholder_analysis(self, stakeholders, network_data):
        """Generate stakeholder analysis section."""
        
        if not stakeholders:
            return "No stakeholder data available."
        
        stakeholder_md = "### Stakeholder Relationships\n\n"
        stakeholder_md += "![Top Stakeholders](../Visualizations/"
        stakeholder_md += f"Top_Stakeholders_{self.politician_name.replace(' ', '_')}.png)\n\n"
        
        # Group stakeholders by type
        people = [s for s in stakeholders if s['type'] == 'person']
        organizations = [s for s in stakeholders if s['type'] == 'organization']
        
        # Top individuals section
        stakeholder_md += "#### Key Individuals in Media Coverage\n\n"
        stakeholder_md += "| Name | Role | Relationship | Mentions | Sentiment |\n"
        stakeholder_md += "|------|------|-------------|----------|----------|\n"
        
        for person in people[:10]:  # Top 10 people
            name = person['name']
            role = person['roles'][0] if person['roles'] else 'Unknown'
            relationship = person['relationship']
            mentions = person['mentions']
            sentiment = f"{person['sentiment_score']:.2f}"
            
            stakeholder_md += f"| {name} | {role} | {relationship} | {mentions} | {sentiment} |\n"
        
        # Top organizations section
        stakeholder_md += "\n#### Key Organizations in Media Coverage\n\n"
        stakeholder_md += "| Organization | Type | Relationship | Mentions | Sentiment |\n"
        stakeholder_md += "|--------------|------|-------------|----------|----------|\n"
        
        for org in organizations[:10]:  # Top 10 organizations
            name = org['name']
            org_type = org['roles'][0] if org['roles'] else 'Unknown'
            relationship = org['relationship']
            mentions = org['mentions']
            sentiment = f"{org['sentiment_score']:.2f}"
            
            stakeholder_md += f"| {name} | {org_type} | {relationship} | {mentions} | {sentiment} |\n"
        
        # Generate AI analysis of stakeholder relationships
        chatbot = ChatGPT(
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=2000
        )
        
        # Prepare top stakeholders for analysis
        top_stakeholders = sorted(stakeholders, key=lambda x: x['mentions'], reverse=True)[:15]
        
        stakeholder_prompt = f"""
Analyze the stakeholder relationships portrayed in media coverage of {self.politician_name} based on the following data:

Top stakeholders:
{json.dumps(top_stakeholders, indent=2)}

Network data:
{json.dumps(network_data, indent=2)}

Provide a comprehensive analysis that:
1. Identifies patterns in how the politician's relationships are portrayed
2. Examines which stakeholders are most frequently associated with the politician and why
3. Analyzes the sentiment patterns in different relationships
4. Discusses potential implications of these relationship portrayals
5. Identifies any clusters or groups of stakeholders with similar characteristics

Structure your analysis with the following sections:
- Overall Relationship Patterns
- Key Political Allies and Opponents
- Institutional Relationships
- Public Perception Implications

Be specific and cite examples from the data where possible.
"""
        
        stakeholder_analysis = chatbot.ask(stakeholder_prompt)
        stakeholder_md += f"\n{stakeholder_analysis}\n\n"
        
        return stakeholder_md


# Main generation functions for integration with MediaCoverageAnalysis.py

def preprocess_politician_articles(politician_name: str, articles: List[Dict], 
                                  news_folder_path: str, force_reprocess: bool = False) -> Tuple[List[Dict], str, bool]:
    """
    Preprocess articles for a politician analysis.
    
    Args:
        politician_name (str): Name of the politician to analyze
        articles (List[Dict]): List of articles to preprocess
        news_folder_path (str): Path to the folder containing news articles
        force_reprocess (bool): Whether to force reprocessing
        
    Returns:
        Tuple[List[Dict], str, bool]: Processed articles, general folder path, and success status
    """
    try:
        # Set up directory structure
        general_folder = setup_politician_directories(news_folder_path, politician_name)
        
        # Filter articles to include only those mentioning the politician
        filtered_articles = []
        
        for article in articles:
            # Check if politician is mentioned in the content
            if politician_name.lower() in article.get('content', '').lower():
                filtered_articles.append(article)
            else:
                # Double check with more sophisticated NLP analysis
                try:
                    chatbot = ChatGPT(
                        model_name="gpt-4o-mini",
                        temperature=0,
                        max_tokens=50
                    )
                    
                    question = f"""
Does this article substantively mention or discuss {politician_name}?
Respond with only "Yes" or "No".

Article: {article.get('content', '')}
                    """
                    
                    response = chatbot.ask(question)
                    if response.strip().lower() == "yes":
                        filtered_articles.append(article)
                except Exception as e:
                    logging.error(f"Error in NLP check: {str(e)}")
                    # If error in NLP check, include by default if keyword is present
                    if politician_name.lower() in (
                        article.get('title', '').lower() + 
                        article.get('author_name', '').lower() + 
                        article.get('media_outlet', '').lower()
                    ):
                        filtered_articles.append(article)
        
        logging.info(f"Filtered {len(filtered_articles)} articles mentioning {politician_name} from {len(articles)} total")
        
        if not filtered_articles:
            logging.warning(f"No articles found mentioning {politician_name}")
            return [], general_folder, False
        
        # Save filtered articles for future use
        articles_path = os.path.join(general_folder, "Outputs", "CompiledOutputs", "FilteredArticles.json")
        with open(articles_path, 'w', encoding='utf-8') as f:
            json.dump(filtered_articles, f, indent=2)
        
        return filtered_articles, general_folder, True
            
    except Exception as e:
        logging.error(f"Error preprocessing politician articles: {str(e)}")
        logging.error(traceback.format_exc())
        return [], "", False

def setup_politician_directories(news_folder_path: str, politician_name: str) -> str:
    """
    Set up directory structure for politician analysis.
    
    Args:
        news_folder_path (str): Path to the folder containing news articles
        politician_name (str): Name of the politician
        
    Returns:
        str: Path to the general folder for this analysis
    """
    general_folder = os.path.dirname(os.path.dirname(news_folder_path))
    
    # Create directory structure
    os.makedirs(os.path.join(general_folder, "Data"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "CompiledOutputs"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis", "PolicyPositions"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis", "Narratives"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis", "Timeline"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "Visualizations"), exist_ok=True)
    
    return general_folder

def generate_politician_profile(politician_name: str, articles_sorted: List[Dict], 
                              general_folder: str, region: str = None,
                              political_party: str = None, language: str = "English",
                              force_reprocess: bool = False) -> str:
    """
    Generate a comprehensive political profile analysis.
    
    Args:
        politician_name (str): Name of the politician to analyze
        articles_sorted (List[Dict]): List of preprocessed articles
        general_folder (str): Path to the general folder for this analysis
        region (str, optional): Geographic region of politician
        political_party (str, optional): Political party affiliation
        language (str, optional): Output language for analysis
        force_reprocess (bool, optional): Whether to force reprocessing
        
    Returns:
        str: Generated markdown content
    """
    try:
        analyzer = PoliticianAnalyzer(
            politician_name=politician_name,
            articles=articles_sorted,
            general_folder=general_folder,
            region=region,
            political_party=political_party,
            language=language,
            force_reprocess=force_reprocess
        )
        
        markdown_content = analyzer.generate_complete_analysis()
        return markdown_content
        
    except Exception as e:
        logging.error(f"Error generating politician profile: {str(e)}")
        logging.error(traceback.format_exc())
        raise

def generate_policy_position_analysis(politician_name: str, articles_sorted: List[Dict],
                                    general_folder: str, policy_focus: str,
                                    language: str = "English") -> str:
    """
    Generate a focused analysis on a specific policy area for a politician.
    
    Args:
        politician_name (str): Name of the politician to analyze
        articles_sorted (List[Dict]): List of preprocessed articles
        general_folder (str): Path to the general folder for this analysis
        policy_focus (str): Specific policy area to analyze
        language (str, optional): Output language for analysis
        
    Returns:
        str: Generated markdown content
    """
    try:
        # Initialize analyzer
        analyzer = PoliticianAnalyzer(
            politician_name=politician_name,
            articles=articles_sorted,
            general_folder=general_folder,
            language=language
        )
        
        # Preprocess articles
        analyzer.preprocess_articles()
        
        # Get all policy areas
        all_policy_areas = analyzer.analyze_policy_positions()
        
        # Find the matching policy area
        matching_policy = None
        for policy in all_policy_areas:
            if policy_focus.lower() in policy['policy_area'].lower():
                matching_policy = policy
                break
        
        if not matching_policy:
            return f"No policy analysis found for '{policy_focus}'. Available policies: {', '.join([p['policy_area'] for p in all_policy_areas])}"
        
        # Collect articles related to this policy
        policy_articles = []
        for article in analyzer.articles:
            if 'policy_areas' in article and matching_policy['policy_area'] in article['policy_areas']:
                policy_articles.append(article)
        
        if not policy_articles:
            return f"No articles found discussing '{policy_focus}' in relation to {politician_name}."
        
        # Generate detailed policy analysis
        chatbot = ChatGPT(
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=3000
        )
        
        policy_prompt = f"""
Generate a comprehensive analysis of how {politician_name}'s positions and statements on {matching_policy['policy_area']} are portrayed in media coverage.

Policy area: {matching_policy['policy_area']}
Description: {matching_policy['description']}
Number of articles: {len(policy_articles)}

Reported positions and statements:
{chr(10).join(matching_policy['positions'])}

Structure your analysis with the following sections:

## {politician_name}'s {matching_policy['policy_area']} Policy Coverage

### Overview
Provide a high-level summary of how this policy area is portrayed.

### Key Positions
Analyze the specific policy positions attributed to the politician, their consistency, and evolution over time.

### Media Framing
Examine how different media outlets frame the politician's stance on this policy.

### Public Reception
Analyze how the public reception of these policy positions is portrayed.

### Context and Comparison
Place these policy positions in the broader political context, including comparisons with other politicians when mentioned.

Conclude with a summary paragraph that captures the overall media narrative around this policy area.
"""
        
        policy_analysis = chatbot.ask(policy_prompt)
        
        # Add article references
        article_references = "\n\n### Sources\n\n"
        for article in policy_articles:
            title = article.get('title', 'Untitled')
            outlet = article.get('media_outlet', 'Unknown')
            date = article.get('date', 'Unknown')
            
            article_references += f"- *{title}* ({outlet}, {date})\n"
        
        final_analysis = f"# Policy Analysis: {politician_name} on {matching_policy['policy_area']}\n\n{policy_analysis}{article_references}"
        
        # Apply translations if needed
        if language.lower() != 'english':
            final_analysis = translate_content(final_analysis, 'auto', language)
        
        # Process citations
        processor = CitationProcessor()
        final_analysis = processor.process_citations_in_markdown(final_analysis)
        
        # Save policy analysis
        output_path = os.path.join(
            general_folder, "Outputs", "PoliticianAnalysis", "PolicyPositions",
            f"Policy_{matching_policy['policy_area'].replace(' ', '_')}_{politician_name.replace(' ', '_')}.md"
        )
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(final_analysis)
        
        return final_analysis
        
    except Exception as e:
        logging.error(f"Error generating policy position analysis: {str(e)}")
        logging.error(traceback.format_exc())
        raise

def generate_stakeholder_relationship_analysis(politician_name: str, articles_sorted: List[Dict],
                                             general_folder: str, stakeholder_focus: str,
                                             language: str = "English") -> str:
    """
    Generate a focused analysis on a specific stakeholder relationship.
    
    Args:
        politician_name (str): Name of the politician to analyze
        articles_sorted (List[Dict]): List of preprocessed articles
        general_folder (str): Path to the general folder for this analysis
        stakeholder_focus (str): Specific stakeholder to analyze
        language (str, optional): Output language for analysis
        
    Returns:
        str: Generated markdown content
    """
    try:
        # Initialize analyzer
        analyzer = PoliticianAnalyzer(
            politician_name=politician_name,
            articles=articles_sorted,
            general_folder=general_folder,
            language=language
        )
        
        # Ensure we have entities extracted
        analyzer.extract_entities()
        
        # Compile stakeholder mentions
        stakeholder_mentions = []
        
        for article in analyzer.articles:
            # Check people entities
            for person in article['entities'].get('people', []):
                if stakeholder_focus.lower() in person['name'].lower():
                    stakeholder_mentions.append({
                        'entity': person,
                        'type': 'person',
                        'article': article
                    })
            
            # Check organization entities
            for org in article['entities'].get('organizations', []):
                if stakeholder_focus.lower() in org['name'].lower():
                    stakeholder_mentions.append({
                        'entity': org,
                        'type': 'organization',
                        'article': article
                    })
        
        if not stakeholder_mentions:
            return f"No mentions found of '{stakeholder_focus}' in relation to {politician_name}."
        
        # Determine the full name of the stakeholder
        most_common_name = Counter([m['entity']['name'] for m in stakeholder_mentions]).most_common(1)[0][0]
        
        # Generate stakeholder relationship analysis
        chatbot = ChatGPT(
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=3000
        )
        
        # Format mentions for analysis
        formatted_mentions = []
        for mention in stakeholder_mentions:
            formatted_mentions.append({
                'name': mention['entity']['name'],
                'type': mention['type'],
                'relationship': mention['entity'].get('relationship', 'Unknown'),
                'context': mention['entity'].get('context', 'Unknown'),
                'article_date': mention['article'].get('date', 'Unknown'),
                'media_outlet': mention['article'].get('media_outlet', 'Unknown'),
                'article_tone': mention['article'].get('tone', 'Neutral'),
                'article_sentiment': mention['article'].get('sentiment_score', 0)
            })
        
        relationship_prompt = f"""
Generate a comprehensive analysis of the relationship between {politician_name} and {most_common_name} as portrayed in media coverage.

Number of mentions: {len(stakeholder_mentions)}
Stakeholder type: {stakeholder_mentions[0]['type']}

Mentions:
{json.dumps(formatted_mentions, indent=2)}

Structure your analysis with the following sections:

## Relationship Analysis: {politician_name} and {most_common_name}

### Relationship Overview
Provide a high-level summary of how the relationship is portrayed.

### Characterization in Media
Analyze how different media outlets characterize this relationship.

### Evolution of the Relationship
Examine how the portrayal of this relationship has evolved over time (if applicable).

### Key Interactions
Highlight significant interactions, agreements, or disagreements mentioned.

### Impact and Significance
Analyze the portrayed impact of this relationship on {politician_name}'s political positions or public image.

Conclude with a summary paragraph that captures the overall media narrative around this relationship.
"""
        
        relationship_analysis = chatbot.ask(relationship_prompt)
        
        # Add article references
        article_references = "\n\n### Sources\n\n"
        seen_articles = set()
        
        for mention in stakeholder_mentions:
            article = mention['article']
            article_id = f"{article.get('title', 'Untitled')}_{article.get('date', 'Unknown')}"
            
            if article_id in seen_articles:
                continue
                
            seen_articles.add(article_id)
            
            title = article.get('title', 'Untitled')
            outlet = article.get('media_outlet', 'Unknown')
            date = article.get('date', 'Unknown')
            
            article_references += f"- *{title}* ({outlet}, {date})\n"
        
        final_analysis = f"# Stakeholder Relationship: {politician_name} and {most_common_name}\n\n{relationship_analysis}{article_references}"
        
        # Apply translations if needed
        if language.lower() != 'english':
            final_analysis = translate_content(final_analysis, 'auto', language)
        
        # Process citations
        processor = CitationProcessor()
        final_analysis = processor.process_citations_in_markdown(final_analysis)
        
        # Save relationship analysis
        output_path = os.path.join(
            general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders",
            f"Relationship_{most_common_name.replace(' ', '_')}_{politician_name.replace(' ', '_')}.md"
        )
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(final_analysis)
        
        return final_analysis
        
    except Exception as e:
        logging.error(f"Error generating stakeholder relationship analysis: {str(e)}")
        logging.error(traceback.format_exc())
        raise

def generate_media_coverage_analysis(politician_name: str, articles_sorted: List[Dict],
                                   general_folder: str, language: str = "English") -> str:
    """
    Generate analysis of how different media outlets cover the politician.
    
    Args:
        politician_name (str): Name of the politician to analyze
        articles_sorted (List[Dict]): List of preprocessed articles
        general_folder (str): Path to the general folder for this analysis
        language (str, optional): Output language for analysis
        
    Returns:
        str: Generated markdown content
    """
    try:
        # Initialize analyzer
        analyzer = PoliticianAnalyzer(
            politician_name=politician_name,
            articles=articles_sorted,
            general_folder=general_folder,
            language=language
        )
        
        # Preprocess articles to ensure we have sentiment
        articles = analyzer.preprocess_articles()
        
        # Group by media outlet
        outlet_articles = defaultdict(list)
        for article in articles:
            outlet = article.get('media_outlet', 'Unknown')
            outlet_articles[outlet].append(article)
        
        # Only analyze outlets with at least 3 articles
        significant_outlets = {outlet: articles for outlet, articles in outlet_articles.items() if len(articles) >= 3}
        
        if not significant_outlets:
            return "No media outlets with sufficient coverage for analysis."
        
        # Calculate statistics for each outlet
        outlet_stats = {}
        for outlet, articles in significant_outlets.items():
            sentiment_scores = [a.get('sentiment_score', 0) for a in articles]
            tones = Counter(a.get('tone', 'Neutral') for a in articles)
            
            # Get narrative categories if available
            narratives = Counter(a.get('narrative_category', 'Unknown') for a in articles if 'narrative_category' in a)
            
            outlet_stats[outlet] = {
                'article_count': len(articles),
                'avg_sentiment': sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0,
                'tone_distribution': {
                    'Positive': tones.get('Positive', 0),
                    'Neutral': tones.get('Neutral', 0),
                    'Negative': tones.get('Negative', 0)
                },
                'top_narratives': narratives.most_common(3)
            }
        
        # Create comparative visualization
        outlets = list(outlet_stats.keys())
        sentiment_values = [outlet_stats[outlet]['avg_sentiment'] for outlet in outlets]
        
        fig, ax = plt.subplots(figsize=(12, 8))
        bars = ax.bar(outlets, sentiment_values)
        
        # Color bars based on sentiment
        for i, bar in enumerate(bars):
            if sentiment_values[i] > 0.5:
                bar.set_color('#4CAF50')  # Green for positive
            elif sentiment_values[i] < -0.5:
                bar.set_color('#F44336')  # Red for negative
            else:
                bar.set_color('#2196F3')  # Blue for neutral
        
        ax.set_title(f'Media Outlet Sentiment Comparison - {politician_name}', fontsize=16)
        ax.set_xlabel('Media Outlet', fontsize=12)
        ax.set_ylabel('Average Sentiment (-5 to 5)', fontsize=12)
        ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        
        chart_path = os.path.join(
            general_folder, "Outputs", "Visualizations",
            f"Media_Comparison_{politician_name.replace(' ', '_')}.png"
        )
        plt.savefig(chart_path, bbox_inches='tight', dpi=300)
        plt.close(fig)
        
        # Generate analysis for each outlet
        media_analysis = f"# Media Coverage Analysis: {politician_name}\n\n"
        media_analysis += "## Comparative Media Analysis\n\n"
        media_analysis += f"![Media Outlet Comparison](../Visualizations/Media_Comparison_{politician_name.replace(' ', '_')}.png)\n\n"
        
        # Generate overall comparative analysis
        chatbot = ChatGPT(
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=2000
        )
        
        comparative_prompt = f"""
Analyze how different media outlets cover {politician_name} based on the following statistics:

{json.dumps(outlet_stats, indent=2)}

Provide an analysis that:
1. Compares and contrasts how different outlets portray the politician
2. Identifies which outlets are most positive/negative and potential reasons why
3. Analyzes differences in narrative focus across outlets
4. Discusses potential implications of these differences in coverage

Structure your analysis with the following sections:
- Overall Patterns
- Sentiment Variations
- Narrative Focus Differences
- Audience and Impact Considerations

Be specific and cite examples from the data where possible.
"""
        
        comparative_analysis = chatbot.ask(comparative_prompt)
        media_analysis += comparative_analysis + "\n\n"
        
        # Add individual outlet analyses
        media_analysis += "## Individual Media Outlet Analysis\n\n"
        
        for outlet, stats in outlet_stats.items():
            outlet_articles = significant_outlets[outlet]
            
            # Take sample articles for analysis
            sample_size = min(5, len(outlet_articles))
            samples = outlet_articles[:sample_size]
            
            # Format for analysis
            sample_data = []
            for article in samples:
                sample_data.append({
                    'title': article.get('title', 'Untitled'),
                    'date': article.get('date', 'Unknown'),
                    'tone': article.get('tone', 'Neutral'),
                    'sentiment_score': article.get('sentiment_score', 0),
                    'narrative_category': article.get('narrative_category', 'Unknown') if 'narrative_category' in article else 'Unknown',
                    'content_excerpt': article.get('content', '')[:500] + "..."  # First 500 chars
                })
            
            outlet_prompt = f"""
Analyze how {outlet} covers {politician_name} based on the following statistics and samples:

Stats:
- Article count: {stats['article_count']}
- Average sentiment: {stats['avg_sentiment']:.2f}
- Tone distribution: {stats['tone_distribution']}
- Top narratives: {stats['top_narratives']}

Article samples:
{json.dumps(sample_data, indent=2)}

Provide a concise (2-paragraph) analysis that:
1. Characterizes this outlet's overall approach to covering the politician
2. Identifies distinctive patterns in language, framing, or focus
3. Compares this outlet's coverage to the average across all outlets

Write your analysis directly, without using headings or section names.
"""
            
            outlet_analysis = chatbot.ask(outlet_prompt)
            media_analysis += f"### {outlet}\n\n"
            media_analysis += f"**Statistics:**\n"
            media_analysis += f"- Articles analyzed: {stats['article_count']}\n"
            media_analysis += f"- Average sentiment: {stats['avg_sentiment']:.2f}\n"
            media_analysis += f"- Tone: {stats['tone_distribution']['Positive']} positive, {stats['tone_distribution']['Neutral']} neutral, {stats['tone_distribution']['Negative']} negative\n\n"
            media_analysis += outlet_analysis + "\n\n"
            
            # Sample articles
            media_analysis += "**Representative articles:**\n\n"
            for i, article in enumerate(samples[:3]):  # List up to 3 examples
                title = article.get('title', 'Untitled')
                date = article.get('date', 'Unknown')
                media_analysis += f"{i+1}. *{title}* ({date})\n"
            
            media_analysis += "\n---\n\n"
        
        # Apply translations if needed
        if language.lower() != 'english':
            media_analysis = translate_content(media_analysis, 'auto', language)
        
        # Process citations
        processor = CitationProcessor()
        media_analysis = processor.process_citations_in_markdown(media_analysis)
        
        # Save media analysis
        output_path = os.path.join(
            general_folder, "Outputs", "CompiledOutputs", 
            f"MediaCoverage_{politician_name.replace(' ', '_')}.md"
        )
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(media_analysis)
        
        return media_analysis
        
    except Exception as e:
        logging.error(f"Error generating media coverage analysis: {str(e)}")
        logging.error(traceback.format_exc())
        raise

def generate_timeline_events_analysis(politician_name: str, articles_sorted: List[Dict],
                                    general_folder: str, language: str = "English") -> str:
    """
    Generate a chronological analysis of events and media coverage.
    
    Args:
        politician_name (str): Name of the politician to analyze
        articles_sorted (List[Dict]): List of preprocessed articles
        general_folder (str): Path to the general folder for this analysis
        language (str, optional): Output language for analysis
        
    Returns:
        str: Generated markdown content
    """
    try:
        # Initialize analyzer
        analyzer = PoliticianAnalyzer(
            politician_name=politician_name,
            articles=articles_sorted,
            general_folder=general_folder,
            language=language
        )
        
        # Get timeline data
        timeline_data = analyzer.create_timeline_analysis()
        
        # Create a horizontal timeline visualization
        months = sorted(timeline_data['timeline_data'].keys())
        counts = [timeline_data['timeline_data'][month] for month in months]
        
        # Format month labels
        month_labels = []
        for month in months:
            try:
                date_obj = datetime.strptime(month, '%Y-%m')
                month_labels.append(date_obj.strftime('%b %Y'))
            except ValueError:
                month_labels.append(month)
        
        # Create a styled figure
        fig, ax = plt.subplots(figsize=(16, 6))
        ax.plot(month_labels, counts, marker='o', linewidth=2, markersize=8, color='#1976D2')
        
        # Add sentiment overlay if available
        if 'monthly_stats' in timeline_data:
            sentiment_values = []
            for month in months:
                if month in timeline_data['monthly_stats']:
                    sentiment_values.append(timeline_data['monthly_stats'][month].get('avg_sentiment', 0))
                else:
                    sentiment_values.append(0)
            
            # Create secondary y-axis for sentiment
            ax2 = ax.twinx()
            ax2.plot(month_labels, sentiment_values, marker='s', linewidth=2, 
                   linestyle='--', color='#FF5722', alpha=0.7, label='Average Sentiment')
            ax2.set_ylabel('Sentiment (-5 to 5)', fontsize=12, color='#FF5722')
            ax2.tick_params(axis='y', colors='#FF5722')
            ax2.set_ylim(-5, 5)
            ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
        
        # Add key events as annotations
        for i, month in enumerate(months):
            if month in timeline_data['key_events'] and timeline_data['key_events'][month]:
                ax.annotate(
                    f"{len(timeline_data['key_events'][month])} events",
                    xy=(i, counts[i]),
                    xytext=(0, 10),
                    textcoords="offset points",
                    ha='center',
                    va='bottom',
                    fontsize=8,
                    bbox=dict(boxstyle="round,pad=0.3", fc="yellow", alpha=0.3)
                )
        
        # Add shift markers
        for shift in timeline_data['shifts']:
            try:
                from_idx = months.index(shift['from_month'])
                to_idx = months.index(shift['to_month'])
                
                if shift['type'] == 'sentiment_shift' and abs(shift['magnitude']) >= 2:
                    # Major sentiment shift
                    ax.annotate(
                        "Major sentiment shift",
                        xy=((from_idx + to_idx) / 2, (counts[from_idx] + counts[to_idx]) / 2),
                        xytext=(0, 20),
                        textcoords="offset points",
                        ha='center',
                        va='bottom',
                        fontsize=8,
                        bbox=dict(boxstyle="round,pad=0.3", fc="red", alpha=0.3),
                        arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2")
                    )
                
                if shift['type'] == 'volume_shift' and (shift['magnitude'] >= 3 or shift['magnitude'] <= 0.33):
                    # Major volume shift
                    ax.annotate(
                        "Major volume shift",
                        xy=((from_idx + to_idx) / 2, (counts[from_idx] + counts[to_idx]) / 2),
                        xytext=(0, -20),
                        textcoords="offset points",
                        ha='center',
                        va='top',
                        fontsize=8,
                        bbox=dict(boxstyle="round,pad=0.3", fc="blue", alpha=0.3),
                        arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2")
                    )
            except (ValueError, IndexError):
                continue
        
        ax.set_title(f'Media Coverage Timeline - {politician_name}', fontsize=16)
        ax.set_xlabel('Time', fontsize=12)
        ax.set_ylabel('Number of Articles', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        
        timeline_chart_path = os.path.join(
            general_folder, "Outputs", "Visualizations",
            f"Detailed_Timeline_{politician_name.replace(' ', '_')}.png"
        )
        plt.savefig(timeline_chart_path, bbox_inches='tight', dpi=300)
        plt.close(fig)
        
        # Generate the timeline analysis
        timeline_analysis = f"# Chronological Analysis: {politician_name}\n\n"
        timeline_analysis += "## Media Coverage Timeline\n\n"
        timeline_analysis += f"![Detailed Timeline](../Visualizations/Detailed_Timeline_{politician_name.replace(' ', '_')}.png)\n\n"
        
        # Generate comprehensive timeline narrative
        chatbot = ChatGPT(
            model_name="chatgpt-4o-latest",
            temperature=0,
            max_tokens=3000
        )
        
        timeline_prompt = f"""
Create a comprehensive chronological narrative of {politician_name}'s media coverage based on this timeline data:

{json.dumps(timeline_data, indent=2)}

Structure your analysis with the following sections:

## Chronological Narrative

### Key Periods and Events
Identify and analyze the most significant periods or events in the coverage.

### Sentiment Evolution
Trace how sentiment toward the politician has evolved over time.

### Narrative Development
Analyze how the framing and narratives about the politician have developed chronologically.

### Coverage Patterns
Identify patterns in the volume and focus of coverage over time.

Be specific about dates, outlets, and events. Format your chronological analysis in a narrative style rather than simply listing events. Focus on explaining WHY shifts in coverage occurred and their significance.
"""
        
        timeline_narrative = chatbot.ask(timeline_prompt)
        timeline_analysis += timeline_narrative + "\n\n"
        
        # Add detailed event timeline
        timeline_analysis += "## Detailed Event Timeline\n\n"
        
        for month in sorted(timeline_data['key_events'].keys()):
            events = timeline_data['key_events'][month]
            if not events:
                continue
            
            try:
                month_display = datetime.strptime(month, '%Y-%m').strftime('%B %Y')
            except ValueError:
                month_display = month
            
            timeline_analysis += f"### {month_display}\n\n"
            
            for event in events:
                title = event.get('title', 'Untitled')
                outlet = event.get('media_outlet', 'Unknown')
                date = event.get('date', 'Unknown')
                tone = event.get('tone', 'Neutral')
                narrative = event.get('narrative_category', 'Unknown')
                
                timeline_analysis += f"**{title}** ({outlet}, {date})\n\n"
                timeline_analysis += f"* Tone: {tone}\n"
                timeline_analysis += f"* Narrative frame: {narrative}\n\n"
        
        # Apply translations if needed
        if language.lower() != 'english':
            timeline_analysis = translate_content(timeline_analysis, 'auto', language)
        
        # Process citations
        processor = CitationProcessor()
        timeline_analysis = processor.process_citations_in_markdown(timeline_analysis)
        
        # Save timeline analysis
        output_path = os.path.join(
            general_folder, "Outputs", "PoliticianAnalysis", "Timeline", 
            f"DetailedTimeline_{politician_name.replace(' ', '_')}.md"
        )
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(timeline_analysis)
        
        return timeline_analysis
        
    except Exception as e:
        logging.error(f"Error generating timeline events analysis: {str(e)}")
        logging.error(traceback.format_exc())
        raise

In [1]:
# Cell 1: Setup and Imports
import os
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Optional
import logging
import traceback
from pathlib import Path

# Import necessary classes from your modules
from Classes.SimplifiedChatbots import ChatGPT, BigSummarizerGPT
from Classes.DocumentProcessor import DocumentProcessor, CitationProcessor
# If you have helpers in Utils folder
# from Utils.Helpers import *

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Cell 2: Configuration Parameters
# Set these parameters according to your needs
PDF_FOLDER_PATH = "KnowledgeBase/Politics/People/AnneAly/NewsAnneAly"  # Replace with your PDF folder path
POLITICIAN_NAME = "Anne Aly"  # Replace with the politician's name
REGION = "Australia"  # E.g., "United States", "European Union", etc.
POLITICAL_PARTY = "Australian Labor Party"  # Optional: political party affiliation
LANGUAGE = "English"  # Language for output
FORCE_REPROCESS = False  # Set to True to force reprocessing of previously analyzed data

In [2]:
# Cell 3: Set up Directory Structure
def setup_politician_directories(news_folder_path, politician_name):
    """
    Set up directory structure for politician analysis.
    Based on the path structure KnowledgeBase/Politics/People/PoliticianName/NewsPoliticianName,
    the general folder should be one level up from the news folder.
    """
    # The general folder is one level up from the news folder
    general_folder = os.path.dirname(news_folder_path)
    
    # Create directory structure
    os.makedirs(os.path.join(general_folder, "Data"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "CompiledOutputs"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis", "PolicyPositions"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis", "Narratives"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "PoliticianAnalysis", "Timeline"), exist_ok=True)
    os.makedirs(os.path.join(general_folder, "Outputs", "Visualizations"), exist_ok=True)
    
    return general_folder

general_folder = setup_politician_directories(PDF_FOLDER_PATH, POLITICIAN_NAME)
print(f"Set up directories in {general_folder}")

# Cell 4: Process PDF Documents to Extract Articles
def process_pdf_documents(pdf_folder, politician_name, political_party=None, region=None):
    """
    Process PDF documents and extract articles relevant to the politician.
    Uses NLP-based filtering for all articles to ensure only articles 
    substantively discussing the politician are included.
    
    Args:
        pdf_folder: Path to folder with PDF files
        politician_name: Name of the politician to focus on
        political_party: Political party of the politician for better filtering
        region: Region/country of the politician for better context
    """
    logging.info(f"Processing PDF documents from {pdf_folder}")
    
    # Initialize document processor
    processor = DocumentProcessor(
        min_length=1000,
        max_length=25500,
        similarity_threshold=0.8
    )
    
    # Process PDFs
    all_articles = processor.process_documents(pdf_folder_path=pdf_folder)
    logging.info(f"Extracted {len(all_articles)} articles from PDFs")
    
    # Add additional context for better filtering
    politician_context = politician_name
    if political_party and region:
        politician_context = f"{politician_name}, a {political_party} politician from {region}"
    elif political_party:
        politician_context = f"{politician_name} of the {political_party}"
    elif region:
        politician_context = f"{politician_name} from {region}"
    
    # Filter articles to include only those that substantively mention the politician
    filtered_articles = []
    
    for i, article in enumerate(all_articles):
        print(f"Analyzing article {i+1}/{len(all_articles)}...")
        
        # Always do NLP-based filtering regardless of keyword match
        try:
            chatbot = ChatGPT(
                model_name="gpt-4o-mini",
                temperature=0,
                max_tokens=50
            )
            
            question = f"""
Does this article substantively discuss {politician_context}? 
Consider the following criteria:
1. The article must discuss the specific politician {politician_name} (not just someone with the same name)
2. The politician should be a focus of discussion, not just mentioned in passing
3. The article should provide meaningful content about the politician's actions, statements, policies, or impact

Respond with only "Yes" or "No".

Article: {article.get('content', '')}
            """
            
            response = chatbot.ask(question)
            if response.strip().lower() == "yes":
                filtered_articles.append(article)
                print(f"✓ Article included: Substantively discusses {politician_name}")
            else:
                print(f"✗ Article excluded: Does not substantively discuss {politician_name}")
                
        except Exception as e:
            logging.error(f"Error in NLP check: {str(e)}")
            # Fallback to basic keyword matching if NLP fails
            if politician_name.lower() in article.get('content', '').lower():
                print(f"? Article included (fallback): Contains keyword '{politician_name}'")
                filtered_articles.append(article)
            else:
                print(f"✗ Article excluded (fallback): No mention of '{politician_name}'")
    
    logging.info(f"Filtered {len(filtered_articles)} articles substantively discussing {politician_name} from {len(all_articles)} total")
    
    # Save filtered articles for future use
    articles_path = os.path.join(general_folder, "Outputs", "CompiledOutputs", "FilteredArticles.json")
    with open(articles_path, 'w', encoding='utf-8') as f:
        json.dump(filtered_articles, f, indent=2)
    
    return filtered_articles

# Process the documents
articles = process_pdf_documents(
    pdf_folder=PDF_FOLDER_PATH, 
    politician_name=POLITICIAN_NAME,
    political_party=POLITICAL_PARTY,
    region=REGION
)
print(f"Processed {len(articles)} articles substantively discussing {POLITICIAN_NAME}")

2025-03-26 12:54:17,119 - root - INFO - Processing PDF documents from KnowledgeBase/Politics/People/AnneAly/NewsAnneAly
2025-03-26 12:54:17,121 - Classes.DocumentProcessor - INFO - Found 93 files in folder


Set up directories in KnowledgeBase/Politics/People/AnneAly


2025-03-26 12:54:17,830 - Classes.DocumentProcessor - INFO - Removing file KnowledgeBase/Politics/People/AnneAly/NewsAnneAly/Australia news live_ ABC managing director hits out at ‘vicious attacks’ on Laura Tingle; Fatima Pay.PDF (text extraction failed or length out of range)
2025-03-26 12:54:18,260 - Classes.DocumentProcessor - INFO - Removing file KnowledgeBase/Politics/People/AnneAly/NewsAnneAly/Sam Mostyn sworn in as governor general – as it happened.PDF (text extraction failed or length out of range)
2025-03-26 12:54:18,658 - Classes.DocumentProcessor - INFO - Removing file KnowledgeBase/Politics/People/AnneAly/NewsAnneAly/Dozens re-detained after visa cancellation bill passes – as it happened.PDF (text extraction failed or length out of range)
2025-03-26 12:54:18,893 - Classes.DocumentProcessor - INFO - Removing file KnowledgeBase/Politics/People/AnneAly/NewsAnneAly/Australia politics live_ Labor chair rejects calls for AFP hearing to be extended amid questions abo.PDF (text ext

Analyzing article 1/46...


2025-03-26 12:54:38,000 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

✓ Article included: Substantively discusses Anne Aly
Analyzing article 45/46...


2025-03-26 12:55:14,448 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✓ Article included: Substantively discusses Anne Aly
Analyzing article 46/46...


2025-03-26 12:55:15,730 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-26 12:55:15,734 - root - INFO - Filtered 39 articles substantively discussing Anne Aly from 46 total


✓ Article included: Substantively discusses Anne Aly
Processed 39 articles substantively discussing Anne Aly


In [5]:
# Cell 5: Create PoliticianAnalyzer Instance
#from Classes.Politician import PoliticianAnalyzer

# Initialize the politician analyzer
analyzer = PoliticianAnalyzer(
    politician_name=POLITICIAN_NAME,
    articles=articles,
    general_folder=general_folder,
    region=REGION,
    political_party=POLITICAL_PARTY,
    language=LANGUAGE,
    force_reprocess=FORCE_REPROCESS
)

# Cell 6: Preprocess Articles for Analysis
# This adds basic sentiment analysis and categorization
preprocessed_articles = analyzer.preprocess_articles()
print(f"Preprocessed {len(preprocessed_articles)} articles with sentiment analysis")

# Cell 7: Policy Position Analysis
# Force reprocessing of policy positions only
policy_clusters = analyzer.analyze_policy_positions(force_reprocess=True)
print(f"Identified {len(policy_clusters)} policy clusters")

# Cell 8: Display Policy Information
# Display the identified policy areas
for i, cluster in enumerate(policy_clusters, 1):
    policy_name = cluster['policy_area']
    position_count = len(cluster.get('positions', []))
    print(f"{i}. {policy_name}: {position_count} positions/statements")
    print(f"   Description: {cluster['description'][:100]}...")
    print()

2025-03-26 13:00:22,354 - root - INFO - Loaded 39 preprocessed articles
2025-03-26 13:00:22,354 - root - INFO - Preprocessing 39 articles
2025-03-26 13:00:22,355 - root - INFO - Using existing sentiment analysis
2025-03-26 13:00:22,355 - root - INFO - Beginning policy position analysis


Preprocessed 39 articles with sentiment analysis
Processing chunk: 1/15


2025-03-26 13:00:26,355 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing chunk: 2/15


2025-03-26 13:00:32,504 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Processing chunk: 15/15


2025-03-26 13:17:12,452 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Extracted positions for Political Safety, Racism & Social Cohesion:
- No specific positions found on Political Safety, Racism & Social Cohesion.

- No specific positions found on Political Safety, Racism & Social Cohesion.

- Anne Aly supported comments by fellow Labor MP Ed Husic, stating that it was "difficult to argue" that Palestinians, including hundreds of children who had been killed, were not being "collectively punished" by Israel.  
  - **Context:** This statement was made in the context of the Israel-Hamas conflict, where some Labor MPs, including Aly, expressed concerns over Israel's blockade on Gaza and its retaliatory attacks. The comments led to accusations of "deep division" within the Albanese government regarding its stance on the conflict.  
  - **Date:** October 19, 2023  
  - **Media outlet:** The Guardian  
  - **Reference:** [The Guardian, Sarah Basford Canales and Paul Karp, October 19, 2023]  

- Prime Minister Anthony Albanese acknowledged that Anne Aly, along

2025-03-26 13:17:18,461 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-26 13:17:18,476 - root - INFO - Saved processing status to KnowledgeBase/Politics/People/AnneAly/Outputs/CompiledOutputs/PoliticianAnalysis_Anne_Aly.json
2025-03-26 13:17:18,477 - root - INFO - Generating policy clusters markdown file for Anne Aly
2025-03-26 13:17:18,480 - root - INFO - Policy clusters markdown file generated at KnowledgeBase/Politics/People/AnneAly/Outputs/PoliticianAnalysis/PolicyPositions/PolicyClusters_Anne_Aly.md
2025-03-26 13:17:20,815 - root - INFO - Policy distribution chart saved to KnowledgeBase/Politics/People/AnneAly/Outputs/Visualizations/PolicyDistribution_Anne_Aly.png


Identified 8 policy clusters
1. Early Childhood Education & Childcare Policy: 70 positions/statements
   Description: This cluster encompasses policies related to early childhood education and childcare, focusing on af...

2. Gender Equality, Workforce Participation & Social Inclusion: 96 positions/statements
   Description: This cluster focuses on policies that promote gender equality, workforce participation, and social i...

3. Housing, Cost of Living & Economic Policy: 62 positions/statements
   Description: This cluster addresses economic policies aimed at improving affordability and financial security for...

4. Youth Engagement, Education & Training: 63 positions/statements
   Description: This cluster focuses on policies aimed at supporting young Australians through education, training, ...

5. National Security, Counterterrorism & Online Safety: 35 positions/statements
   Description: This cluster covers policies related to national security, counterterrorism, and online safet

In [ ]:
# Initialize the politician analyzer
analyzer = PoliticianAnalyzer(
    politician_name=POLITICIAN_NAME,
    articles=articles,
    general_folder=general_folder,
    region=REGION,
    political_party=POLITICAL_PARTY,
    language=LANGUAGE,
    force_reprocess=FORCE_REPROCESS
)

2025-03-26 18:59:18,141 - root - INFO - Loaded 39 preprocessed articles
2025-03-26 18:59:18,142 - root - INFO - Generating policy clusters markdown file for Anne Aly
2025-03-26 18:59:20,035 - root - INFO - Policy distribution chart saved to KnowledgeBase/Politics/People/AnneAly/Outputs/Visualizations/PolicyDistribution_Anne_Aly.png
2025-03-26 18:59:25,260 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-26 18:59:28,963 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-26 18:59:28,976 - root - INFO - Successfully processed citations to superscript format
2025-03-26 18:59:28,978 - root - INFO - Policy clusters markdown file generated at KnowledgeBase/Politics/People/AnneAly/Outputs/PoliticianAnalysis/PolicyPositions/PolicyClusters_Anne_Aly.md


In [ ]:
# Cell 9: Analyze Stakeholders and Relationships
# This analyzes the politician's relationships with various stakeholders mentioned in the articles

print(f"Starting stakeholder analysis for {POLITICIAN_NAME}...")
stakeholder_data = analyzer.analyze_stakeholders()
print(f"✓ Stakeholder analysis completed!")

# Overview of results
num_stakeholders = len(stakeholder_data["stakeholders"])
num_categories = len(stakeholder_data["categories"])
print(f"\nStakeholder Analysis Summary:")
print(f"- Identified {num_stakeholders} unique stakeholders with connections to {POLITICIAN_NAME}")
print(f"- Categorized stakeholders into {num_categories} relationship groups")

# Display the top stakeholders by mention count
top_stakeholders = sorted(stakeholder_data["stakeholders"], key=lambda x: x['mentions'], reverse=True)[:10]
print("\nTop 10 Stakeholders:")
for i, stakeholder in enumerate(top_stakeholders, 1):
    name = stakeholder['name']
    mentions = stakeholder['mentions']
    role = stakeholder['role']
    relationship = stakeholder['relationship_type']
    category = stakeholder['category']
    stakeholder_type = stakeholder['type']
    
    print(f"{i}. {name} ({stakeholder_type})")
    print(f"   Category: {category}")
    print(f"   Role: {role}")
    print(f"   Relationship: {relationship}")
    print(f"   Mentions: {mentions}")
    print()

# Stakeholder categories overview
print("\nStakeholder Categories:")
for category in stakeholder_data["categories"]:
    category_name = category["category"]
    stakeholders_in_category = [s for s in stakeholder_data["stakeholders"] if s['category'] == category_name]
    print(f"- {category_name}: {len(stakeholders_in_category)} stakeholders")

# Path to detailed report
report_path = os.path.join(
    general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders",
    f"StakeholderAnalysisReport_{POLITICIAN_NAME.replace(' ', '_')}.md"
)
print(f"\nDetailed stakeholder analysis report saved to: {report_path}")

2025-03-26 18:59:42,420 - root - INFO - Beginning stakeholder analysis for Anne Aly
2025-03-26 18:59:42,422 - root - INFO - Extracting stakeholders from article 1/39


Starting stakeholder analysis for Anne Aly...


2025-03-26 18:59:48,681 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-26 18:59:48,685 - root - INFO - Extracted 7 stakeholders from article 1
2025-03-26 18:59:48,686 - root - INFO - Extracting stakeholders from article 2/39


```json
[
    {
        "name": "Anne Aly",
        "type": "person",
        "description": "Federal Early Childhood Education Minister responsible for policies related to early childhood education and care in Australia."
    },
    {
        "name": "Jay Weatherill",
        "type": "person",
        "description": "Former Premier of South Australia and current leader of the early education campaign Thrive by Five, advocating for reforms in early childhood education."
    },
    {
        "name": "Georgie Dent",
        "type": "person",
        "description": "Chief Executive of The Parenthood, a campaign partner of Thrive by Five, focused on making early childhood education more affordable for families."
    },
    {
        "name": "Nesha Hutchinson",
        "type": "person",
        "description": "Vice-President of the Australian Childcare Alliance, representing childcare providers and advocating for their interests."
    },
    {
        "name": "Emily Holmes",
        "type":

2025-03-26 18:59:56,209 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-26 18:59:56,213 - root - INFO - Extracted 9 stakeholders from article 2
2025-03-26 18:59:56,214 - root - INFO - Extracting stakeholders from article 3/39


...

2025-03-26 19:17:24,808 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# **Stakeholder Mapping Analysis: Personal and Cultural Influences on Anne Aly**  

## **1. Power-Interest Analysis**  

### **Stakeholder Power and Interest Assessment**  

| **Stakeholder**            | **Power** | **Interest** | **Assessment** |
|----------------------------|----------|-------------|---------------|
| **Leila Naja Hibri**       | Medium   | High        | As CEO of the Australian Fashion Council, she has influence in the fashion and sustainability sectors, aligning with Aly’s interests. |
| **Thomas Puttick**         | Low      | High        | A designer promoting sustainable fashion, he shares Aly’s values but has limited political power. |
| **Edith Cowan (Legacy)**   | High     | High        | As a historical figure, her legacy is symbolically powerful, reinforcing Aly’s role as a trailblazer. |
| **Islamophobia Register Australia** | High | High | This organization plays a crucial role in supporting Aly’s advocacy against discrimination. |
| **Barack Obama**     

2025-03-26 19:17:25,298 - root - INFO - Stakeholder analysis report saved to KnowledgeBase/Politics/People/AnneAly/Outputs/PoliticianAnalysis/Stakeholders/StakeholderAnalysisReport_Anne_Aly.md
2025-03-26 19:17:25,303 - root - INFO - Beginning stakeholder analysis for Anne Aly
2025-03-26 19:17:25,304 - root - INFO - Loading existing stakeholder analysis


✓ Stakeholder analysis completed!

Stakeholder Analysis Summary:
- Identified 160 unique stakeholders with connections to Anne Aly
- Categorized stakeholders into 9 relationship groups

Top 10 Stakeholders:
1. Anne Aly (person)
   Category: Early Childhood Education and Advocacy Groups
   Role: Federal Early Childhood Education Minister
   Relationship: policy influencer
   Mentions: 28

2. Anthony Albanese (person)
   Category: Government and Political Colleagues
   Role: Prime Minister of Australia
   Relationship: colleague
   Mentions: 20

3. Ed Husic (person)
   Category: Government and Political Colleagues
   Role: Labor frontbencher and member of the Australian Parliament
   Relationship: colleague
   Mentions: 12

4. Labor Party (organization)
   Category: Government and Political Colleagues
   Role: Minister for Early Childhood Education
   Relationship: ally
   Mentions: 11

5. Penny Wong (person)
   Category: Foreign Policy and International Relations
   Role: Foreign Minist

TypeError: string indices must be integers, not 'str'

In [39]:
# Cell 10: Rerun just the stakeholder category analyses with updated prompting
import json
import os
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import logging

# Initialize the politician analyzer
analyzer = PoliticianAnalyzer(
    politician_name=POLITICIAN_NAME,
    articles=articles,
    general_folder=general_folder,
    region=REGION,
    political_party=POLITICAL_PARTY,
    language=LANGUAGE,
    force_reprocess=FORCE_REPROCESS
)

# Load the existing stakeholder data
stakeholder_path = os.path.join(
    general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders",
    f"StakeholderAnalysis_{POLITICIAN_NAME.replace(' ', '_')}.json"
)

with open(stakeholder_path, 'r', encoding='utf-8') as f:
    stakeholder_data = json.load(f)

# Extract the components we want to keep
stakeholders = stakeholder_data["stakeholders"]
categories = stakeholder_data["categories"]

# Initialize dictionary to store new analyses
updated_category_analyses = {}

print(f"Rerunning stakeholder category analyses for {POLITICIAN_NAME}...")

# Perform category analyses with updated prompting
for category in categories:
    category_name = category["category"]
    stakeholders_in_category = [s for s in stakeholders if s['category'] == category_name]
    
    if not stakeholders_in_category:
        updated_category_analyses[category_name] = {
            "category_info": category,
            "stakeholders": [],
            "analysis": "No stakeholders identified in this category."
        }
        continue
    
    # Format stakeholder data for analysis
    stakeholder_data_for_analysis = []
    for s in stakeholders_in_category:
        stakeholder_data_for_analysis.append({
            "name": s['name'],
            "type": s['type'],
            "role": s['role'],
            "relationship_type": s['relationship_type'],
            "mentions": s['mentions'],
            "description": s['description'],
            "influence_description": s.get('influence_description', '')
        })
    
    # Use your updated prompt here
    chatbot = ChatGPT(
        model_name="chatgpt-4o-latest",
        temperature=0,
        max_tokens=2000
    )
    # Load category_info


    # YOUR CUSTOM PROMPT GOES HERE
    analysis_prompt = f"""
Perform a comprehensive stakeholder mapping analysis for the following group of stakeholders related to {POLITICIAN_NAME}.

These stakeholders belong to the category: "{category_name}"
Category description: {category["description"]}

Stakeholders in this category:
{json.dumps(stakeholder_data, indent=2)}

Your analysis should include:

1. Power-Interest Analysis:
    - Identify which stakeholders have high/low power and high/low interest regarding {POLITICIAN_NAME}.
    - Explain the basis for these power and interest assessments.
    - Use these columns to format this part of the assessment: | Stakeholder | Power | Interest | Basis for Assessment |

2. Combined Network and Influence Analysis:
    - Provide an integrated analysis that merges network mapping with a detailed influence assessment.
    - Identify key network connectors and central influencers as well as any isolated or peripheral stakeholders, highlighting how these relationships could lead to alliances or conflicts.
    - Describe the communication flows and information exchange dynamics within the network.
    - Clearly pinpoint specific stakeholders capable of exerting direct influence on {POLITICIAN_NAME}, detailing the nature of their influence (supportive, adversarial, or conditional) and explaining the factors and conditions that might shift their stance.
    - Format this section using a combination of narrative paragraphs and concise, descriptive bullet points that deliver rich information without excessive listing.
    - Do not include any concluding statement, stop straight after the last point of the section.

Format your response as a professionally written, comprehensive analysis with clear sections and insights. Start at the '###' header level as your output will be embedded in a markdown document. Use a mixture of bullet points and paragraphs to format your output. Do not use '---' symbols as delimiters.

Your output should only be the following two sections and nothing more: ###Power-Interest Analysis, ###Combined Network and Influence Analysis. No introduction or conclusion needed. Do not even include any concluding statement, stop straight after the last point in the Combined Network and Influence Analysis section.
"""

    
    analysis_response = chatbot.ask(analysis_prompt)
    print(f"✓ Completed analysis for category: {category_name}")
    print(analysis_response)
    
    updated_category_analyses[category_name] = {
        "category_info": category,
        "stakeholders": stakeholders_in_category,
        "analysis": analysis_response
    }

# Update the stakeholder data with new analyses
updated_stakeholder_data = {
    "stakeholders": stakeholders,
    "categories": categories,
    "category_analyses": updated_category_analyses
}

# Save the updated results
with open(stakeholder_path, 'w', encoding='utf-8') as f:
    json.dump(updated_stakeholder_data, f, indent=2)

print("✓ Updated stakeholder analyses saved")

# Regenerate the markdown report
# Note: This code assumes analyzer._generate_stakeholder_analysis_report exists
print("Regenerating stakeholder analysis report...")
analyzer._generate_stakeholder_analysis_report(updated_stakeholder_data)

# Path to detailed report
report_path = os.path.join(
    general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders",
    f"StakeholderAnalysisReport_{POLITICIAN_NAME.replace(' ', '_')}.md"
)
print(f"✓ Updated stakeholder analysis report saved to: {report_path}")

2025-03-27 15:58:29,812 - root - INFO - Loaded 39 preprocessed articles


Rerunning stakeholder category analyses for Anne Aly...


2025-03-27 15:59:02,491 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✓ Completed analysis for category: Government and Political Colleagues
### Power-Interest Analysis

| **Stakeholder** | **Power** | **Interest** | **Basis for Assessment** |
|---------------|---------|---------|----------------------|
| **Anthony Albanese (Prime Minister)** | High | High | As the head of government, he has significant influence over policy direction and funding, directly impacting Anne Aly’s initiatives. |
| **Jason Clare (Minister for Education)** | High | High | Directly collaborates with Anne Aly on education policies, making him a key partner in policy implementation. |
| **Jim Chalmers (Treasurer)** | High | High | Controls budget allocations, which affect funding for early childhood education initiatives. |
| **Labor Party** | High | High | Provides political backing and policy direction for Anne Aly’s initiatives. |
| **Albanese Government** | High | High | The governing body that sets the overall policy agenda, influencing Anne Aly’s work. |
| **Productivity Co

2025-03-27 16:00:05,375 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

✓ Completed analysis for category: Labor and Workforce Advocacy
### Power-Interest Analysis

| **Stakeholder** | **Power** | **Interest** | **Basis for Assessment** |
|---------------|---------|---------|----------------------|
| **Fair Work Commission** | High | High | As the regulatory body overseeing multi-employer wage negotiations, the Fair Work Commission has significant authority in shaping labor policies, including those affecting childcare workers. Its interest is high due to its role in enforcing participation in wage deals, directly impacting Anne Aly’s workforce policies. |
| **Independent Education Union (IEU)** | Medium | High | Represents educators and childcare workers, giving it strong advocacy influence. While it lacks direct regulatory power, its ability to mobilize workers and influence public discourse makes it a key ally for Aly’s labor rights agenda. |
| **United Workers Union** | Medium | High | A major labor union advocating for early childhood educators, influ

2025-03-27 16:06:33,768 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✓ Completed analysis for category: Personal and Cultural Influences
### Power-Interest Analysis

| Stakeholder | Power | Interest | Basis for Assessment |
|------------|-------|----------|----------------------|
| **Leila Naja Hibri (CEO, Australian Fashion Council)** | Medium | High | As a leader in the fashion industry, she has influence in sustainability and cultural advocacy, aligning with Aly’s interests. |
| **Thomas Puttick (Sustainable Fashion Designer)** | Low | High | A designer promoting sustainable fashion, he shares Aly’s values but has limited political power. |
| **Edith Cowan (Symbolic Legacy)** | High | High | As the first female politician in Australia, her legacy is symbolically powerful, reinforcing Aly’s role as a trailblazer. |
| **Islamophobia Register Australia** | High | High | This organization plays a crucial role in supporting Aly’s advocacy against discrimination and Islamophobia. |
| **Barack Obama (Former U.S. President)** | High | Medium | As a global po

2025-03-27 16:06:34,187 - root - INFO - Stakeholder appendix report saved to KnowledgeBase/Politics/People/AnneAly/Outputs/PoliticianAnalysis/Stakeholders/StakeholderAnalysisAppendix_Anne_Aly.md
2025-03-27 16:06:34,189 - root - INFO - Stakeholder analysis report saved to KnowledgeBase/Politics/People/AnneAly/Outputs/PoliticianAnalysis/Stakeholders/StakeholderAnalysisReport_Anne_Aly.md


✓ Updated stakeholder analysis report saved to: KnowledgeBase/Politics/People/AnneAly/Outputs/PoliticianAnalysis/Stakeholders/StakeholderAnalysisReport_Anne_Aly.md


In [32]:
# Initialize the politician analyzer
analyzer = PoliticianAnalyzer(
    politician_name=POLITICIAN_NAME,
    articles=articles,
    general_folder=general_folder,
    region=REGION,
    political_party=POLITICAL_PARTY,
    language=LANGUAGE,
    force_reprocess=FORCE_REPROCESS
)

# Note: This code assumes analyzer._generate_stakeholder_analysis_report exists
print("Regenerating stakeholder analysis report...")
analyzer._generate_stakeholder_analysis_report(updated_stakeholder_data)

# Path to detailed report
report_path = os.path.join(
    general_folder, "Outputs", "PoliticianAnalysis", "Stakeholders",
    f"StakeholderAnalysisReport_{POLITICIAN_NAME.replace(' ', '_')}.md"
)
print(f"✓ Updated stakeholder analysis report saved to: {report_path}")

2025-03-27 14:58:56,122 - root - INFO - Loaded 39 preprocessed articles


Regenerating stakeholder analysis report...


2025-03-27 14:58:56,649 - root - INFO - Stakeholder analysis report saved to KnowledgeBase/Politics/People/AnneAly/Outputs/PoliticianAnalysis/Stakeholders/StakeholderAnalysisReport_Anne_Aly.md


✓ Updated stakeholder analysis report saved to: KnowledgeBase/Politics/People/AnneAly/Outputs/PoliticianAnalysis/Stakeholders/StakeholderAnalysisReport_Anne_Aly.md
